<a href="https://colab.research.google.com/github/descobarsalce/AutomaticTradingAgent/blob/main/Modeling/1_1_Modeling_Data_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This script has the objective of preparing data for the model and it has two steps:

1. It reads the waste generation data and prepares it for the models, extracting some variables such as point of measurement and generators and preprocessing.

2. It then reads the independent variables dataset with all states/counties and prepares it for the models.

# Setup

In [2]:
keep_imputations = False # Whether to keep the imputations for potential generators or to only use the original values (which include most observations as "Unspecified" category).
small_sample = False # Whether to keep only the sample with know generators or to also keep the "Unspecified" category.
generate_interactions_variables = False # These are variables designed to generate "generator-specific explanatory variables". For example, to force institutional markers variables to only be called into the function for observations containing "Institutional" waste.

name_suffix = 'KeepImp0_GenInter0_SmallSample0' # suffix to add when saving files

input_date = '02_04_2025'
output_date = '02_04_2025'

folder_path_RAW              = '/content/drive/MyDrive/Waste_Prediction_GITHUB_v2/WasteModeling/' # The raw files are in the independent variables dataset.
folder_path_INPUT_VARIABLES  = '/content/drive/MyDrive/Waste_Prediction_Outputs/Ind_Vars_Output/' + input_date + '/' # Which version of the independent variables to call
folder_path_OUTPUT_VARIABLES = '/content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/' + output_date + '/'  # Which version of the independent variables to call


## Imports

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.makedirs(folder_path_OUTPUT_VARIABLES, exist_ok=True)

In [5]:
# Time just to track the duration of the script run and the time of the last update.
import pytz
from datetime import datetime

# Define Central Time timezone
central_time = pytz.timezone('US/Central')

# Get the current time in UTC and convert it to Central Time
current_time_central = datetime.now(central_time)
print("Central Time:", current_time_central.strftime('%Y-%m-%d %H:%M:%S %Z'))

import pandas as pd
import pickle
import numpy as np
import re
import sys
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import (train_test_split, cross_val_score, GridSearchCV,
                                  ParameterGrid, StratifiedKFold, StratifiedShuffleSplit,KFold)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import (DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_graphviz)
from sklearn.ensemble import (GradientBoostingRegressor, GradientBoostingClassifier,
                          RandomForestRegressor, RandomForestClassifier)
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score,
                          explained_variance_score, mean_absolute_percentage_error,
                          accuracy_score, f1_score, precision_score, recall_score,
                          roc_auc_score, precision_recall_curve, average_precision_score,
                          confusion_matrix, classification_report)
from IPython.display import Image
from tqdm.auto import tqdm
import logging
import pickle
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.utils.multiclass import type_of_target
import graphviz
import json

Central Time: 2025-02-10 08:32:33 CST


# Data Loading:

## Y variables:

### Loading generation data:

In [6]:
df_Y_county = pd.read_csv(folder_path_RAW + 'Dependent Variables/Waste Generation/postAgg_fips.csv')
print(df_Y_county.shape)
df_Y_county

(2338, 18)


,DF Name,ActivityProducedBy,ActivityConsumedBy,Year,Point of Measurement,TemporalDataQuality,GeographicDataQuality,CompletenessDataQuality,DataReliability,FlowName,Waste Generation Units,Recycling Included,Disposal Included,Yard Waste Included,Food Waste Included,Waste Generation Amount,Location,FIPS
0,df62_0_0,Unspecified MSW,Landfill,2012,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,15330.0,12043,S12043
1,df62_0_0,Unspecified MSW,Landfill,2013,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,6893.0,12077,S12077
2,df62_0_0,Unspecified MSW,Landfill,2012,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,7127.0,12077,S12077
3,df62_0_0,Unspecified MSW,Landfill,2014,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,40187.0,12043,S12043
4,df62_0_0,Unspecified MSW,Landfill,2013,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,16872.0,12043,S12043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,df432_0_0,Unspecified MSW,"Landfill, Recycling",2019,Curbside Collection,1,1,1,5,municipal solid waste,tons,N,Y,U,Y,178370.4,36085,S36085
2334,df432_0_0,Unspecified MSW,"Landfill, Recycling",2020,Curbside Collection,1,1,1,5,municipal solid waste,tons,N,Y,U,Y,200321.4,36085,S36085
2335,df432_0_0,Unspecified MSW,"Landfill, Recycling",2021,Curbside Collection,1,1,1,5,municipal solid waste,tons,N,Y,U,Y,198159.8,36085,S36085
2336,df432_0_0,Unspecified MSW,"Landfill, Recycling",2022,Curbside Collection,1,1,1,5,municipal solid waste,tons,N,Y,U,Y,182057.8,36085,S36085


In [7]:
df_Y_states = pd.read_excel(folder_path_RAW + 'Dependent Variables/Waste Generation/State_ValidationData.xlsx')
df_Y_states.drop(columns=['County', 'County-Level OR All Municipalities in County Included', 'Municipality/City'], inplace=True)

# The data needs to be shifted to our identifiers "FIPS-Year", instead of state name and year:
df_state_county_fips = pd.read_csv(folder_path_RAW + 'Independent Variables/Source Files/state_and_county_fips.csv')
df_state_county_fips = df_state_county_fips[df_state_county_fips.FIPS.str.endswith('000')]
df_state_county_fips = df_state_county_fips[['state','FIPS']].drop_duplicates().sort_values(by='FIPS')
print(df_state_county_fips.shape)
df_state_county_fips

df_Y_states = df_Y_states.merge(df_state_county_fips, left_on='State', right_on='state', how='left').drop(columns='state')
df_Y_states.rename(columns={'State': 'Location'}, inplace=True)
print(df_Y_states.shape)
df_Y_states

(52, 2)
(78, 18)


,DF Name,Location,ActivityProducedBy,ActivityConsumedBy,Year,Point of Measurement,TemporalDataQuality,GeographicDataQuality,CompletenessDataQuality,DataReliability,FlowName,Waste Generation Amount,Waste Generation Units,Recycling Included,Disposal Included,Yard Waste Included,Food Waste Included,FIPS
0,df105_0_0,Indiana,"Unspecified Residential, Commercial, Instituti...",Landfill,2008,Landfill,1,1,1,1,municipal solid waste,5996960.00,tons,N,Y,U,U,S18000
1,df116_0_0,Kansas,Unspecified MSW,Landfill,2010,Landfill,1,3,5,1,municipal solid waste,2924190.00,tons,N,Y,U,U,S20000
2,df116_0_0,Kansas,Unspecified MSW,Landfill,2011,Landfill,1,3,5,1,municipal solid waste,2779197.00,tons,N,Y,U,U,S20000
3,df116_0_0,Kansas,Unspecified MSW,Landfill,2012,Landfill,1,3,5,1,municipal solid waste,2719214.00,tons,N,Y,U,U,S20000
4,df116_0_0,Kansas,Unspecified MSW,Landfill,2013,Landfill,1,3,5,1,municipal solid waste,2806949.00,tons,N,Y,U,U,S20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,df99_0_0,Illinois,Unspecified MSW,Landfill,2016,Landfill,1,3,5,1,municipal solid waste,16741185.71,tons,N,Y,U,U,S17000
74,df99_0_0,Illinois,Unspecified MSW,Landfill,2017,Landfill,1,3,5,1,municipal solid waste,17024149.92,tons,N,Y,U,U,S17000
75,df99_0_0,Illinois,Unspecified MSW,Landfill,2018,Landfill,1,3,5,1,municipal solid waste,16887071.93,tons,N,Y,U,U,S17000
76,df99_0_0,Illinois,Unspecified MSW,Landfill,2019,Landfill,1,3,5,1,municipal solid waste,16350896.97,tons,N,Y,U,U,S17000


In [8]:
df_Y = pd.concat([df_Y_county, df_Y_states], ignore_index=True, axis=0)
print(df_Y.shape)
print(df_Y.columns)
df_Y

(2416, 18)
Index(['DF Name', 'ActivityProducedBy', 'ActivityConsumedBy', 'Year',
       'Point of Measurement', 'TemporalDataQuality', 'GeographicDataQuality',
       'CompletenessDataQuality', 'DataReliability', 'FlowName',
       'Waste Generation Units', 'Recycling Included', 'Disposal Included',
       'Yard Waste Included', 'Food Waste Included', 'Waste Generation Amount',
       'Location', 'FIPS'],
      dtype='object')


,DF Name,ActivityProducedBy,ActivityConsumedBy,Year,Point of Measurement,TemporalDataQuality,GeographicDataQuality,CompletenessDataQuality,DataReliability,FlowName,Waste Generation Units,Recycling Included,Disposal Included,Yard Waste Included,Food Waste Included,Waste Generation Amount,Location,FIPS
0,df62_0_0,Unspecified MSW,Landfill,2012,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,15330.00,12043,S12043
1,df62_0_0,Unspecified MSW,Landfill,2013,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,6893.00,12077,S12077
2,df62_0_0,Unspecified MSW,Landfill,2012,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,7127.00,12077,S12077
3,df62_0_0,Unspecified MSW,Landfill,2014,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,40187.00,12043,S12043
4,df62_0_0,Unspecified MSW,Landfill,2013,Unspecified Waste Facility,1,1,5,5,municipal solid waste,tons,Y,Y,U,U,16872.00,12043,S12043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,df99_0_0,Unspecified MSW,Landfill,2016,Landfill,1,3,5,1,municipal solid waste,tons,N,Y,U,U,16741185.71,Illinois,S17000
2412,df99_0_0,Unspecified MSW,Landfill,2017,Landfill,1,3,5,1,municipal solid waste,tons,N,Y,U,U,17024149.92,Illinois,S17000
2413,df99_0_0,Unspecified MSW,Landfill,2018,Landfill,1,3,5,1,municipal solid waste,tons,N,Y,U,U,16887071.93,Illinois,S17000
2414,df99_0_0,Unspecified MSW,Landfill,2019,Landfill,1,3,5,1,municipal solid waste,tons,N,Y,U,U,16350896.97,Illinois,S17000


### Data merge (1 case):

In [9]:
# Combining df46_0_0 Single and Multi-Family Residential because no other study makes the distinction. This is the only case where this is reported/tracked separately.
print(df_Y[df_Y['ActivityProducedBy'] == 'Single Family Residential'].shape) # just 1 observation
print(df_Y[df_Y['ActivityProducedBy'] == 'Multi-Family Residential'].shape) # just 1 observation as well
df46_0_0_single = df_Y[df_Y['ActivityProducedBy'] == 'Single Family Residential']
df46_0_0_multi = df_Y[df_Y['ActivityProducedBy'] == 'Multi-Family Residential']

if not df46_0_0_single.empty and not df46_0_0_multi.empty:
    single_index = df46_0_0_single.index[0]
    multi_index = df46_0_0_multi.index[0]
    # We replace the observation indexed as "single-family" and relabel it as general "Residential"; then replace the generation amount with the total including single+multi family, and drop the multi-family observation.
    df_Y.at[single_index, 'ActivityProducedBy'] = 'Residential'
    sum_single_multi = df_Y.at[single_index, 'Waste Generation Amount'] + df_Y.at[multi_index, 'Waste Generation Amount']
    df_Y.at[single_index, 'Waste Generation Amount'] = sum_single_multi
    df_Y.drop(multi_index, inplace=True)

(1, 18)
(1, 18)


### Data tabulations:

In [10]:
print(df_Y.columns)

Index(['DF Name', 'ActivityProducedBy', 'ActivityConsumedBy', 'Year',
       'Point of Measurement', 'TemporalDataQuality', 'GeographicDataQuality',
       'CompletenessDataQuality', 'DataReliability', 'FlowName',
       'Waste Generation Units', 'Recycling Included', 'Disposal Included',
       'Yard Waste Included', 'Food Waste Included', 'Waste Generation Amount',
       'Location', 'FIPS'],
      dtype='object')


In [11]:
df_Y[[ 'Recycling Included', 'Disposal Included', 'Yard Waste Included', 'Food Waste Included']].value_counts(dropna=False)

Recycling Included  Disposal Included  Yard Waste Included  Food Waste Included
Y                   Y                  U                    U                      1453
N                   Y                  U                    U                       696
                                                            Y                       120
Y                   Y                  Y                    Y                        99
N                   Y                  Y                    U                        15
                                                            Y                        11
Y                   Y                  N                    U                        10
                                       Y                    U                         9
                                       N                    Y                         1
                                       Y                    N                         1
Name: count, dtype: int64

In [12]:
df_Y[['ActivityConsumedBy']].value_counts(dropna=False)

,count
ActivityConsumedBy,
Landfill,1320
"Landfill, Recycling",849
"Landfill, MRF",47
"Landfill,Recycling",45
"Landfill, Incinerator, Recycling",23
"Landfill, Incinerator",21
"Recycling, Compost,Landfill, Incinerator",16
"Landfilled, Recycling",13
"Landfill, Recycling, Incinerator",13


In [13]:
df_Y[['ActivityProducedBy']].value_counts(dropna=False)

,count
ActivityProducedBy,
Unspecified MSW,1373
"Commercial, Residential, Institutional, Industrial",492
"Unspecified MSW, Industrial, C&D",260
"Unspecified Residential, Commercial",116
"Unspecified Residential, Commercial, Institutional",39
"Unspecified Residential, Commercial, Industrial",26
"Commercial, Unspecified Residential",24
Unspecified Residential,16
Commercial,14


In [14]:
df_Y[['ActivityProducedBy', 'ActivityConsumedBy']].value_counts(dropna=False)

ActivityProducedBy                                                   ActivityConsumedBy                                
Unspecified MSW                                                      Landfill                                              698
                                                                     Landfill, Recycling                                   582
Commercial, Residential, Institutional, Industrial                   Landfill                                              492
Unspecified MSW, Industrial, C&D                                     Landfill, Recycling                                   260
Unspecified Residential, Commercial                                  Landfill                                              103
Unspecified MSW                                                      Landfill, MRF                                          47
Unspecified Residential, Commercial, Institutional                   Recycling, Compost,Landfill, Incinerator               16
Unspecified Residential, Commercial, Industrial                      Landfilled, Recycling                                  12
Commercial, Unspecified Residential                                  Landfill, Incinerator, Recycling                       12
Unspecified MSW                                                      Landfill, Incinerator                                  12
Unspecified Residential                                              Landfill,Recycling                                     10
Unspecified MSW                                                      Landfill, Recycling, Composting, Incinerator           10
                                                                     Landfill, Incinerator, Recycling                       10
Unspecified Residential, Commercial, Institutional                   Landfill                                                9
Unspecified Residential, Commercial, Industrial                      Landfill, Incinerator,Recycling, Compost                8
Commercial, Unspecified Residential, Institutional, C&D              Landfill,Compost, Recycling                             8
Commercial                                                           Landfill, Recycling, Incinerator                        7
Unspecified Residential, Commercial, Institutional                   Landfill,Recycling                                      7
Commercial                                                           Landfill,Recycling                                      6
Commercial, Unspecified Residential                                  Recycling, Composting,Landfill                          6
Unspecified Residential, Commercial, Industrial                      Landfill                                                6
Commercial, Unspecified Residential                                  Landfill, Incinerator                                   6
Unspecified Residential, Commercial, Institutional                   Landfill,Diverted                                       5
Unspecified Residential, Institutional, Commercial, Industrial       Landfill                                                5
Unspecified Residential, Comercial                                   Recycling, Composting, Landfill                         5
Unspecified MSW                                                      Landfill,Recycling                                      5
Commercial, Institutional                                            Landfill,Recycling                                      5
Unspecified Residential, Institutional, Commercial                   Landfill,Recycling                                      5
Unspecified Residential, Commercial, Landscape, C&D                  Recycling,Landfill                                      5
Unspecified Residential, Commercial, C&D                             Landfill, Recycling                                     5
Unspecified Residential                                              L

In [15]:
df_Y[['Point of Measurement', 'ActivityConsumedBy']].value_counts(dropna=False)

Point of Measurement                                      ActivityConsumedBy                                
Unspecified Waste Facility                                Landfill, Recycling                                   729
                                                          Landfill                                              691
Landfill                                                  Landfill                                              628
Curbside Collection                                       Landfill, Recycling                                   120
Landfill, MRF                                             Landfill, MRF                                          47
Unspecified Waste Facility                                Landfill,Recycling                                     25
                                                          Landfill, Incinerator, Recycling                       22
                                                          Landfill, Incinerator                                  21
                                                          Recycling, Compost,Landfill, Incinerator               16
                                                          Landfilled, Recycling                                  13
                                                          Landfill, Recycling, Incinerator                       13
                                                          Landfill, Recycling, Composting, Incinerator           11
                                                          Landfill,Recycling, Composting                          8
Landfill,Recycling                                        Landfill,Recycling                                      8
Unspecified Waste Facility                                Landfill, Incinerator,Recycling, Compost                8
Landfill,Compost, Recycling                               Landfill,Compost, Recycling                             8
Landfill,Unspecified Waste Facility                       Landfill,Recycling                                      7
Unspecified Waste Facility                                Recycling, Composting, Landfill                         6
Transfer Station,Material Recovery Facility               Landfill,Recycling                                      5
Unspecified Waste Facility                                Recycling,Landfill                                      5
Landfill,Unspecified Waste Facility                       Landfill,Diverted                                       5
Landfill,Recycling,Composting,Unspecified Waste Facility  Landfill,Recycling,Yard Waste, Incinerator,Compost      4
Unspecified Waste Facility                                Unknown                                                 3
Waste Haulers, Self-Haul                                  Recycling, Composting,Landfill                          3
Transfer Station                                          Recycling, Composting,Landfill                          3
Landfill                                                  Landfill, Incinerator, Recycling                        1
Unspecified Waste Facility                                Landfill, Incinerator,Recycling                         1
Landfill,Incinerator/Combustion                           Landfill,Incinerator/Combustion                         1
Landfill,Recycling                                        Landfill                                                1
Unspecified Waste Facility                                Compost, Recycling,Landfill, Incinerator                1
                                                          Landfill, Incinerator,Recycling, Composting             1
Name: count, dtype: int64

In [16]:
df_Y[['TemporalDataQuality', 'GeographicDataQuality', 'CompletenessDataQuality', 'DataReliability']].value_counts(dropna=False)

TemporalDataQuality  GeographicDataQuality  CompletenessDataQuality  DataReliability
1                    1                      5                        5                  669
                                            1                        1                  640
                     5                      5                        1                  466
                     3                      5                        4                  264
                     1                      1                        5                  167
                                            5                        1                  111
                     3                      5                        1                   40
                     1                      5                        2                   11
                     3                      1                        4                   11
                     5                      5                        2                   11
                     3                      1                        1                    7
                     1                      1                        2                    6
                     5                      3                        4                    6
                                            5                        5                    6
Name: count, dtype: int64

In [17]:
# Check if there are any 5 in any of the values:
df_Y[['TemporalDataQuality', 'GeographicDataQuality', 'CompletenessDataQuality', 'DataReliability']].isin([5]).sum()



,0
TemporalDataQuality,0
GeographicDataQuality,489
CompletenessDataQuality,1578
DataReliability,842


In [18]:
df_Y[['CompletenessDataQuality']].value_counts(dropna=False)

,count
CompletenessDataQuality,
5,1578
1,831
3,6


In [19]:
df_Y[['GeographicDataQuality']].value_counts(dropna=False)

,count
GeographicDataQuality,
1,1604
5,489
3,322


In [20]:
df_Y[['DataReliability']].value_counts(dropna=False)

,count
DataReliability,
1,1264
5,842
4,281
2,28


In [21]:
df_Y[['FlowName']].value_counts(dropna=False)

,count
FlowName,
municipal solid waste,2415


In [22]:
pd.DataFrame(df_Y['DF Name'].value_counts(dropna=False, ascending=False))

,count
DF Name,
df62_0_0,670
df432_0_0,120
df109_0_0,92
df167_1_0,82
df167_0_0,82
...,...
df119_0_0,1
df38_0_0,1
df104_0_0,1


### Variables generation:

In [23]:
# Homogeneizing strings to create ActivityConsumedBy categories as separate variables
df_Y['ActivityConsumedBy'] = df_Y['ActivityConsumedBy'].str.replace('Composting', 'Compost')
df_Y['ActivityConsumedBy'] = df_Y['ActivityConsumedBy'].str.replace('Incinerator/Combustion', 'Incinerator')
df_Y['ActivityConsumedBy'] = df_Y['ActivityConsumedBy'].str.replace('Landfilled', 'Landfill')
# df_Y['ActivityConsumedBy'] = df_Y['ActivityConsumedBy'].str.replace(',Yard Waste', ', Yard Waste')
df_Y['ActivityConsumedBy'] = df_Y['ActivityConsumedBy'].str.replace('Diverted', 'Recycling')
df_Y['ActivityConsumedBy'] = df_Y['ActivityConsumedBy'].str.replace('Comercial', 'Commercial')

#Parsing out the comma-delimited list in ActivityConsumedBy to create dummy variables
df_Y['ActivityConsumedBy'] = df_Y['ActivityConsumedBy'].apply(lambda x: ','.join([item.strip() for item in x.split(',')]))
df_Y = df_Y.join(df_Y['ActivityConsumedBy'].str.get_dummies(sep=',').add_prefix('ActivityConsumedBy_'))

#Parsing out the comma-delimited list in ActivityProducedBy to create dummy variables
df_Y['ActivityProducedBy'] = df_Y['ActivityProducedBy'].str.replace('Unspecified Residential', 'Residential')
df_Y['ActivityProducedBy'] = df_Y['ActivityProducedBy'].str.replace('Comercial', 'Commercial')
df_Y['ActivityProducedBy'] = df_Y['ActivityProducedBy'].apply(lambda x: ','.join([item.strip() for item in x.split(',')]))
df_Y = df_Y.join(df_Y['ActivityProducedBy'].str.get_dummies(sep=',').add_prefix('ActivityProducedBy_'))

df_Y.rename(columns={'Point of Measurement': 'PointOfMeasurement'}, inplace=True)
df_Y['ActivityConsumedBy'] = df_Y['ActivityConsumedBy'].str.replace('Composting', 'Compost')
df_Y['PointOfMeasurement'] = df_Y['PointOfMeasurement'].apply(lambda x: ','.join([item.strip() for item in x.split(',')]))
df_Y = df_Y.join(df_Y['PointOfMeasurement'].str.get_dummies(sep=',').add_prefix('PointOfMeasurement_'))

# Display new variables with binary indicators for each category:
df_Y[['ActivityConsumedBy_Compost',
       'ActivityConsumedBy_Incinerator', 'ActivityConsumedBy_Landfill',
       'ActivityConsumedBy_MRF', 'ActivityConsumedBy_Recycling',
       'ActivityConsumedBy_Unknown', 'ActivityConsumedBy_Yard Waste',
       'ActivityProducedBy_C&D',
       'ActivityProducedBy_Commercial', 'ActivityProducedBy_Industrial',
       'ActivityProducedBy_Institutional', 'ActivityProducedBy_Landscape',
       'ActivityProducedBy_Residential', 'ActivityProducedBy_Unspecified MSW',
       'PointOfMeasurement_Compost', 'PointOfMeasurement_Composting',
       'PointOfMeasurement_Curbside Collection',
       'PointOfMeasurement_Incinerator/Combustion',
       'PointOfMeasurement_Landfill', 'PointOfMeasurement_MRF',
       'PointOfMeasurement_Material Recovery Facility',
       'PointOfMeasurement_Recycling', 'PointOfMeasurement_Self-Haul',
       'PointOfMeasurement_Transfer Station',
       'PointOfMeasurement_Unspecified Waste Facility',
       'PointOfMeasurement_Waste Haulers']]

,ActivityConsumedBy_Compost,ActivityConsumedBy_Incinerator,ActivityConsumedBy_Landfill,ActivityConsumedBy_MRF,ActivityConsumedBy_Recycling,ActivityConsumedBy_Unknown,ActivityConsumedBy_Yard Waste,ActivityProducedBy_C&D,ActivityProducedBy_Commercial,ActivityProducedBy_Industrial,...,PointOfMeasurement_Curbside Collection,PointOfMeasurement_Incinerator/Combustion,PointOfMeasurement_Landfill,PointOfMeasurement_MRF,PointOfMeasurement_Material Recovery Facility,PointOfMeasurement_Recycling,PointOfMeasurement_Self-Haul,PointOfMeasurement_Transfer Station,PointOfMeasurement_Unspecified Waste Facility,PointOfMeasurement_Waste Haulers
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2412,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2413,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2414,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [24]:
# Duplicate information deleted and maintained only in "Yard Waste Included"
print(pd.crosstab(df_Y['Yard Waste Included'],df_Y['ActivityConsumedBy_Yard Waste']))
df_Y.drop('ActivityConsumedBy_Yard Waste', axis=1, inplace=True)

ActivityConsumedBy_Yard Waste     0  1
Yard Waste Included                   
N                                11  0
U                              2269  0
Y                               131  4


In [25]:
print(df_Y[[ 'Recycling Included', 'Disposal Included', 'Yard Waste Included', 'Food Waste Included']].value_counts(dropna=False))

df_Y['Recycling Included'] = np.where(df_Y['Recycling Included'] == "Y", 1, np.where(df_Y['Recycling Included'] == "N", 0, np.nan))
df_Y['Disposal Included'] = np.where(df_Y['Disposal Included'] == "Y", 1, np.where(df_Y['Disposal Included'] == "N", 0, np.nan))

df_Y['Yard Waste Included Unknown'] = np.where(df_Y['Yard Waste Included'] == "U", 1, 0)
df_Y['Yard Waste Included'] = np.where(df_Y['Yard Waste Included'] == "Y", 1, 0) # Unknowns are coded the same as "N" but there is another variable to differentiate them

df_Y['Food Waste Included Unknown'] = np.where(df_Y['Food Waste Included'] == "U", 1, 0)
df_Y['Food Waste Included'] = np.where(df_Y['Food Waste Included'] == "Y", 1, 0)

print(df_Y[['Recycling Included', 'Disposal Included', 'Yard Waste Included', 'Yard Waste Included Unknown', 'Food Waste Included', 'Food Waste Included Unknown']].value_counts(dropna=False))

Recycling Included  Disposal Included  Yard Waste Included  Food Waste Included
Y                   Y                  U                    U                      1453
N                   Y                  U                    U                       696
                                                            Y                       120
Y                   Y                  Y                    Y                        99
N                   Y                  Y                    U                        15
                                                            Y                        11
Y                   Y                  N                    U                        10
                                       Y                    U                         9
                                       N                    Y                         1
                                       Y                    N                         1
Name: count, dtype: int64
Recycling Incl

In [26]:
df_Y.rename(columns={'FIPS': 'FIPS_2020'}, inplace=True)

In [27]:
df_Y.isna().sum().sort_values(ascending=False)

,0
DF Name,0
ActivityConsumedBy_Unknown,0
ActivityProducedBy_Commercial,0
ActivityProducedBy_Industrial,0
ActivityProducedBy_Institutional,0
ActivityProducedBy_Landscape,0
ActivityProducedBy_Residential,0
ActivityProducedBy_Unspecified MSW,0
PointOfMeasurement_Compost,0
PointOfMeasurement_Composting,0


### Saving:

In [28]:
df_Y.to_csv(folder_path_OUTPUT_VARIABLES + 'PostProcessingWasteGenDependentVariables.csv', index=False)
print(f"File saved to path:" + folder_path_OUTPUT_VARIABLES + 'PostProcessingWasteGenDependentVariables.csv')
df_Y

File saved to path:/content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/PostProcessingWasteGenDependentVariables.csv


,DF Name,ActivityProducedBy,ActivityConsumedBy,Year,PointOfMeasurement,TemporalDataQuality,GeographicDataQuality,CompletenessDataQuality,DataReliability,FlowName,...,PointOfMeasurement_Landfill,PointOfMeasurement_MRF,PointOfMeasurement_Material Recovery Facility,PointOfMeasurement_Recycling,PointOfMeasurement_Self-Haul,PointOfMeasurement_Transfer Station,PointOfMeasurement_Unspecified Waste Facility,PointOfMeasurement_Waste Haulers,Yard Waste Included Unknown,Food Waste Included Unknown
0,df62_0_0,Unspecified MSW,Landfill,2012,Unspecified Waste Facility,1,1,5,5,municipal solid waste,...,0,0,0,0,0,0,1,0,1,1
1,df62_0_0,Unspecified MSW,Landfill,2013,Unspecified Waste Facility,1,1,5,5,municipal solid waste,...,0,0,0,0,0,0,1,0,1,1
2,df62_0_0,Unspecified MSW,Landfill,2012,Unspecified Waste Facility,1,1,5,5,municipal solid waste,...,0,0,0,0,0,0,1,0,1,1
3,df62_0_0,Unspecified MSW,Landfill,2014,Unspecified Waste Facility,1,1,5,5,municipal solid waste,...,0,0,0,0,0,0,1,0,1,1
4,df62_0_0,Unspecified MSW,Landfill,2013,Unspecified Waste Facility,1,1,5,5,municipal solid waste,...,0,0,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,df99_0_0,Unspecified MSW,Landfill,2016,Landfill,1,3,5,1,municipal solid waste,...,1,0,0,0,0,0,0,0,1,1
2412,df99_0_0,Unspecified MSW,Landfill,2017,Landfill,1,3,5,1,municipal solid waste,...,1,0,0,0,0,0,0,0,1,1
2413,df99_0_0,Unspecified MSW,Landfill,2018,Landfill,1,3,5,1,municipal solid waste,...,1,0,0,0,0,0,0,0,1,1
2414,df99_0_0,Unspecified MSW,Landfill,2019,Landfill,1,3,5,1,municipal solid waste,...,1,0,0,0,0,0,0,0,1,1


## X variables:

In [29]:
df_predictors = pd.read_parquet(folder_path_INPUT_VARIABLES + '5C+S-AllIndVars_filled_RENAMED.parquet')
df_predictors

,FIPS_2020,Year,DEM: Population,DEM: Median Household Income,State,FIPS_before_map,County,County_2020,DEM: Proportion Below Poverty Line,DEM: Unemployment Rate,...,"IP: Other services, except government",IP: Federal government enterprises (GFE),IP: Federal general government (defense - GFGD),IP: Federal general government (nondefense - GFGN),IP: State and local government enterprises (GSLE),IP: State and local general government (GSLG),IP: Housing (531 real estate),IP: Other real estate (531 - not housing),Combo,count_reps
0,S41011,2018,63308.0,43308.000000,Oregon,S41011,[Coos County],Coos,0.171000,0.077000,...,0.009049,0.000753,0.002265,0.001319,0.010889,0.048389,0.013481,0.007901,None,NaN
1,S36089,2018,109558.0,49305.000000,New york,S36089,[St. Lawrence County],St. Lawrence,0.177000,0.074000,...,0.012342,0.000728,0.004247,0.001274,0.014366,0.063840,0.019929,0.011680,None,NaN
2,S18029,2002,47052.0,51194.000000,Indiana,S18029,[Dearborn],Dearborn,0.069981,0.063299,...,0.004730,0.000656,0.001246,0.001219,0.003871,0.015197,0.003902,0.001644,None,NaN
3,S01037,2017,10955.0,34792.000000,Alabama,S01037,[Coosa County],Coosa,0.144000,0.113000,...,0.007847,0.001251,0.011656,0.001802,0.007866,0.035197,0.009097,0.004185,None,NaN
4,S48153,2012,6439.0,35323.000000,Texas,S48153,[Floyd County],Floyd,0.232000,0.040000,...,0.005521,0.000678,0.005330,0.001277,0.005537,0.024437,0.005408,0.002832,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73446,S56000,2018,581836.0,67311.945455,Wyoming,S56000,None,NA,0.111889,0.045263,...,0.005286,0.000766,0.007307,0.001341,0.010435,0.046372,0.008414,0.004932,S56000_2018,23.0
73447,S56000,2019,581024.0,68939.696970,Wyoming,S56000,None,NA,0.110094,0.045246,...,0.005176,0.000699,0.007252,0.001001,0.009641,0.042951,0.007719,0.004705,S56000_2019,23.0
73448,S56000,2020,581348.0,70567.448485,Wyoming,S56000,None,NA,0.108014,0.044309,...,0.005246,0.000753,0.007872,0.001295,0.010429,0.046409,0.009203,0.005047,S56000_2020,23.0
73449,S56000,2021,576641.0,72195.200000,Wyoming,S56000,None,NA,0.107858,0.042353,...,0.005459,0.000716,0.008497,0.001035,0.010967,0.048938,0.009904,0.005490,S56000_2021,23.0


## Merge X and Y:

In [30]:
df_full = pd.merge(df_Y, df_predictors, on=['Year', 'FIPS_2020'], indicator='merge_predictors', how='outer')
print(df_full.merge_predictors.value_counts()) # There are 7 with merge=left_only. They are from 2023.
print("--------------------------------")
print(df_full.columns)
print("--------------------------------")
print(df_full.shape)
df_full =df_full[df_full.merge_predictors!='left_only'] # XXX CHECK the observations from 2023 do not appear in the predictors dataset so they are dropped.
df_full = df_full[['Year', 'FIPS_2020'] + [col for col in df_full.columns if col not in ['Year', 'FIPS_2020']]]
df_full

merge_predictors
right_only    71074
both           2408
left_only         7
Name: count, dtype: int64
--------------------------------
Index(['DF Name', 'ActivityProducedBy', 'ActivityConsumedBy', 'Year',
       'PointOfMeasurement', 'TemporalDataQuality', 'GeographicDataQuality',
       'CompletenessDataQuality', 'DataReliability', 'FlowName',
       ...
       'IP: Federal government enterprises (GFE)',
       'IP: Federal general government (defense - GFGD)',
       'IP: Federal general government (nondefense - GFGN)',
       'IP: State and local government enterprises (GSLE)',
       'IP: State and local general government (GSLG)',
       'IP: Housing (531 real estate)',
       'IP: Other real estate (531 - not housing)', 'Combo', 'count_reps',
       'merge_predictors'],
      dtype='object', length=288)
--------------------------------
(73489, 288)


,Year,FIPS_2020,DF Name,ActivityProducedBy,ActivityConsumedBy,PointOfMeasurement,TemporalDataQuality,GeographicDataQuality,CompletenessDataQuality,DataReliability,...,IP: Federal government enterprises (GFE),IP: Federal general government (defense - GFGD),IP: Federal general government (nondefense - GFGN),IP: State and local government enterprises (GSLE),IP: State and local general government (GSLG),IP: Housing (531 real estate),IP: Other real estate (531 - not housing),Combo,count_reps,merge_predictors
1,2000,S01000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002895,0.003938,0.005355,0.008145,0.032441,0.005416,0.002219,S01000_2000,23.0,right_only
2,2000,S01001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002895,0.003938,0.005355,0.008145,0.032441,0.005416,0.002219,None,NaN,right_only
3,2000,S01003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002895,0.003938,0.005355,0.008145,0.032441,0.005416,0.002219,None,NaN,right_only
4,2000,S01005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002895,0.003938,0.005355,0.008145,0.032441,0.005416,0.002219,None,NaN,right_only
5,2000,S01007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002895,0.003938,0.005355,0.008145,0.032441,0.005416,0.002219,None,NaN,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73478,2022,S56037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000698,0.009093,0.000978,0.011233,0.050376,0.010322,0.005759,None,NaN,right_only
73479,2022,S56039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000698,0.009093,0.000978,0.011233,0.050376,0.010322,0.005759,None,NaN,right_only
73480,2022,S56041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000698,0.009093,0.000978,0.011233,0.050376,0.010322,0.005759,None,NaN,right_only
73481,2022,S56043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000698,0.009093,0.000978,0.011233,0.050376,0.010322,0.005759,None,NaN,right_only


### Saving predictors out of sample (requires all dataset):

In [31]:
df_full['original_index'] = df_full.index # Mostly for verification

We hold on saving this dataset until after applying the scaling.

### Training data:

In [32]:
# For training purposes we only keep the data that has generation output
updated_df_train = df_full[(~df_full['Waste Generation Amount'].isna()) & (df_full['Waste Generation Amount']!=0)] # XXX CHECK: there are a few with generation=0 that do not make sense

# This data only has missing values in the generators and the county name (state level data without county)
pd.DataFrame(updated_df_train.isna().sum().sort_values(ascending=False))

updated_df_train

,Year,FIPS_2020,DF Name,ActivityProducedBy,ActivityConsumedBy,PointOfMeasurement,TemporalDataQuality,GeographicDataQuality,CompletenessDataQuality,DataReliability,...,IP: Federal general government (defense - GFGD),IP: Federal general government (nondefense - GFGN),IP: State and local government enterprises (GSLE),IP: State and local general government (GSLG),IP: Housing (531 real estate),IP: Other real estate (531 - not housing),Combo,count_reps,merge_predictors,original_index
211,2000,S06037,df431_0_0,Unspecified MSW,"Landfill,MRF","Landfill,MRF",1.0,1.0,1.0,5.0,...,0.009897,0.004027,0.010853,0.041523,0.009794,0.003516,None,NaN,both,211
1807,2000,S34001,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,None,NaN,both,1807
1808,2000,S34003,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,None,NaN,both,1808
1809,2000,S34005,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,None,NaN,both,1809
1810,2000,S34007,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,None,NaN,both,1810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72174,2022,S36047,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,None,NaN,both,72174
72181,2022,S36061,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,None,NaN,both,72181
72191,2022,S36081,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,None,NaN,both,72191
72193,2022,S36085,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,None,NaN,both,72193


#### Create weights for observations.

These weights are based on aiming to represent equally each state, study, county, and population.

In [33]:
# import pandas as pd
# import numpy as np
# from scipy import stats
# import seaborn as sns
# import matplotlib.pyplot as plt
# from typing import List, Dict
# from tabulate import tabulate
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.ensemble import GradientBoostingClassifier

# def generate_weights_via_logistic_regression(
#     full_dataset: pd.DataFrame,
#     training_subset: pd.DataFrame,
#     categorical_cols: List[str],
#     continuous_cols: List[str],
#     aggressiveness: float = 1.0,
#     target_col: str = None,
# ) -> pd.Series:
#     """
#     Generate weights for the training subset to make it more representative of the full dataset.
#     Uses a Gradient Boosting Classifier to estimate the probability of belonging to the full dataset vs training subset.
#     The 'aggressiveness' parameter adjusts how strongly the model tries to correct for differences.

#     Parameters:
#     - full_dataset: pd.DataFrame, the full dataset
#     - training_subset: pd.DataFrame, the training subset
#     - categorical_cols: List[str], list of feature column names that are categorical
#     - continuous_cols: List[str], list of feature column names that are continuous
#     - aggressiveness: float, controls the aggressiveness of weighting (default is 1.0)
#     - target_col: str, optional, the target variable to ensure balanced representation

#     Returns:
#     - pd.Series of weights for the training subset
#     """
#     feature_cols = categorical_cols + continuous_cols
#     # Validate inputs
#     if not all(col in full_dataset.columns for col in feature_cols):
#         raise ValueError("All feature columns must exist in full dataset")
#     if not all(col in training_subset.columns for col in feature_cols):
#         raise ValueError("All feature columns must exist in training subset")

#     # Initialize weights
#     base_weights = np.ones(len(training_subset))

#     # Combine datasets and create labels (1 for full dataset, 0 for training subset)
#     combined = pd.concat([full_dataset[feature_cols], training_subset[feature_cols]], axis=0, ignore_index=True)
#     labels = np.concatenate([np.ones(len(full_dataset)), np.zeros(len(training_subset))])

#     # Create and fit preprocessor
#     transformers = []
#     if continuous_cols:
#         transformers.append(('num', StandardScaler(), continuous_cols))
#     if categorical_cols:
#         transformers.append(('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols))

#     if transformers:
#         preprocessor = ColumnTransformer(transformers)
#         combined_features = preprocessor.fit_transform(combined)
#     else:
#         # If no features to transform, use the data as is
#         combined_features = combined.values

#     # Adjust model parameters based on aggressiveness
#     # Higher aggressiveness leads to stronger corrections
#     learning_rate = 0.1 * aggressiveness
#     n_estimators = int(100 * aggressiveness)
#     max_depth = int(3 * aggressiveness)

#     # Ensure parameters are within reasonable bounds
#     learning_rate = max(min(learning_rate, 1.0), 0.01)
#     n_estimators = max(n_estimators, 10)
#     max_depth = max(int(max_depth), 1)

#     # Fit Gradient Boosting Classifier
#     model = GradientBoostingClassifier(
#         n_estimators=n_estimators,
#         learning_rate=learning_rate,
#         max_depth=max_depth,
#         random_state=42
#     )
#     model.fit(combined_features, labels)

#     # Calculate propensity scores
#     if transformers:
#         training_features = preprocessor.transform(training_subset[feature_cols])
#     else:
#         training_features = training_subset[feature_cols].values
#     propensity_scores = model.predict_proba(training_features)[:, 1]

#     # Clip propensity scores to avoid division by zero or infinity weights
#     epsilon = 1e-6
#     propensity_scores = np.clip(propensity_scores, epsilon, 1 - epsilon)

#     # Calculate inverse probability weights
#     weights = propensity_scores / (1 - propensity_scores)

#     # Adjust weights based on aggressiveness
#     weights = weights ** aggressiveness

#     # Handle extreme weights
#     weight_cap = 10.0 ** aggressiveness  # Increase cap with aggressiveness
#     weights = np.clip(weights, 0.1, weight_cap)  # Limit weight range

#     # Combine with base weights from stratification (if any)
#     final_weights = weights * base_weights

#     # Normalize weights to have mean of 1
#     final_weights /= np.mean(final_weights)

#     return pd.Series(final_weights, index=training_subset.index)

# def get_weighted_value_counts(data: pd.Series, weights: pd.Series) -> pd.Series:
#     """Calculates weighted value counts for categorical data."""
#     df = pd.DataFrame({'data': data, 'weights': weights})
#     weighted_counts = df.groupby('data')['weights'].sum()
#     weighted_dist = weighted_counts / weighted_counts.sum()
#     return weighted_dist

# def plot_categorical_distribution_heatmap(
#     full_data: pd.Series, train_data: pd.Series, weights: pd.Series, col_name: str
# ):
#     """Plots categorical distribution using heatmaps for better representation of all categories."""
#     categories = full_data.unique()

#     # If col_name is 'Population_Bin', sort categories based on bin lower bounds
#     if col_name == 'Population_Bin':
#         # Extract lower bounds from bin labels
#         def get_lower_bound(bin_label):
#             lower_str = bin_label.split('-')[0]
#             return float(lower_str)
#         categories = sorted(categories, key=get_lower_bound)

#     full_counts = full_data.value_counts(normalize=True).reindex(categories, fill_value=0)
#     train_counts = train_data.value_counts(normalize=True).reindex(categories, fill_value=0)
#     weighted_counts = get_weighted_value_counts(train_data, weights).reindex(categories, fill_value=0)

#     df_plot = pd.DataFrame({
#         'Full Dataset': full_counts,
#         'Training (Unweighted)': train_counts,
#         'Training (Weighted)': weighted_counts
#     }).T

#     # Now df_plot columns (categories) are sorted
#     plt.figure(figsize=(12, 6))
#     sns.heatmap(df_plot, annot=True, cmap='YlOrRd', cbar=True, fmt='.2f')
#     plt.title(f'Heatmap of Distribution for {col_name}')
#     plt.ylabel('Dataset')
#     plt.xlabel('Category')
#     plt.xticks(rotation=45, ha='right')
#     plt.tight_layout()

# def plot_weighted_histogram(data: pd.Series, weights: np.ndarray, label: str):
#     """Plots a weighted histogram."""
#     plt.hist(data, bins=30, weights=weights, alpha=0.5, label=label, density=True)
#     plt.xlabel(data.name)
#     plt.ylabel('Frequency')

# def plot_empirical_cdf(data: pd.Series, weights: np.ndarray, label: str):
#     """Plots the empirical CDF for weighted data."""
#     sorted_indices = np.argsort(data)
#     sorted_data = data.iloc[sorted_indices]
#     sorted_weights = weights[sorted_indices]
#     cumulative_weights = np.cumsum(sorted_weights)
#     cumulative_weights /= cumulative_weights[-1]
#     plt.step(sorted_data, cumulative_weights, label=label)
#     plt.xlabel(data.name)
#     plt.ylabel('Cumulative Probability')

# def calculate_js_divergence(p: pd.Series, q: pd.Series) -> float:
#     """Calculates Jensen-Shannon divergence between two distributions."""
#     p = p.reindex(q.index).fillna(0)
#     q = q.reindex(p.index).fillna(0)
#     m = 0.5 * (p + q)
#     return 0.5 * (stats.entropy(p, m, base=2) + stats.entropy(q, m, base=2))

# def calculate_improvement_percentage(
#     full_dist: pd.Series, train_dist: pd.Series, weighted_dist: pd.Series
# ) -> float:
#     """Calculates percentage improvement in distribution similarity after weighting."""
#     unweighted_div = calculate_js_divergence(full_dist, train_dist)
#     weighted_div = calculate_js_divergence(full_dist, weighted_dist)
#     if unweighted_div == 0:
#         return 0.0
#     return ((unweighted_div - weighted_div) / unweighted_div) * 100

# def calculate_weighted_ks_statistic(
#     full_data: np.ndarray, train_data: np.ndarray, full_weights: np.ndarray, train_weights: np.ndarray
# ) -> float:
#     """Calculates weighted Kolmogorov-Smirnov statistic."""
#     # Combine data and weights
#     data = np.concatenate([full_data, train_data])
#     weights = np.concatenate([full_weights, train_weights])
#     labels = np.concatenate([np.zeros(len(full_data)), np.ones(len(train_data))])

#     # Sort data
#     sorted_indices = np.argsort(data)
#     data_sorted = data[sorted_indices]
#     weights_sorted = weights[sorted_indices]
#     labels_sorted = labels[sorted_indices]

#     # Calculate weighted cumulative sums
#     cumsum_full = np.cumsum(weights_sorted * (labels_sorted == 0)) / full_weights.sum()
#     cumsum_train = np.cumsum(weights_sorted * (labels_sorted == 1)) / train_weights.sum()

#     # KS statistic
#     ks_statistic = np.max(np.abs(cumsum_full - cumsum_train))
#     return ks_statistic

# def calculate_distribution_metrics(
#     full_dataset: pd.DataFrame,
#     training_subset: pd.DataFrame,
#     weights: pd.Series,
#     categorical_cols: List[str],
#     continuous_cols: List[str]
# ) -> Dict:
#     """Calculates comprehensive distribution similarity metrics."""
#     metrics = {
#         'sample_sizes': {
#             'full_dataset': len(full_dataset),
#             'training_subset': len(training_subset),
#             'effective_sample_size': (weights.sum() ** 2) / (weights ** 2).sum()
#         },
#         'categorical_metrics': {},
#         'continuous_metrics': {}
#     }

#     # Categorical metrics
#     for col in categorical_cols:
#         full_dist = full_dataset[col].value_counts(normalize=True)
#         train_dist = training_subset[col].value_counts(normalize=True)
#         weighted_dist = get_weighted_value_counts(training_subset[col], weights)

#         metrics['categorical_metrics'][col] = {
#             'unweighted_js_divergence': calculate_js_divergence(full_dist, train_dist),
#             'weighted_js_divergence': calculate_js_divergence(full_dist, weighted_dist),
#             'improvement_percentage': calculate_improvement_percentage(
#                 full_dist, train_dist, weighted_dist
#             )
#         }

#     # Continuous metrics
#     for col in continuous_cols:
#         log_full_data = np.log1p(full_dataset[col])
#         log_training_data = np.log1p(training_subset[col])
#         metrics['continuous_metrics'][col] = {
#             'ks_test_unweighted': stats.ks_2samp(
#                 log_full_data, log_training_data
#             ).statistic,
#             'ks_test_weighted': calculate_weighted_ks_statistic(
#                 log_full_data.values,
#                 log_training_data.values,
#                 np.ones(len(full_dataset)),
#                 weights.values
#             )
#         }

#     return metrics

# def display_results_table(metrics: Dict):
#     """Displays the metrics in a tabular format."""
#     table = []
#     for col, col_metrics in metrics['categorical_metrics'].items():
#         table.append([
#             col,
#             f"{col_metrics['unweighted_js_divergence']:.4f}",
#             f"{col_metrics['weighted_js_divergence']:.4f}",
#             f"{col_metrics['improvement_percentage']:.1f}%"
#         ])
#     headers = ["Variable", "Unweighted JS Divergence", "Weighted JS Divergence", "Improvement"]
#     print(tabulate(table, headers=headers, tablefmt="grid"))

#     # Continuous metrics
#     for col, col_metrics in metrics['continuous_metrics'].items():
#         print(f"\nContinuous Variable: {col}")
#         print(f"Unweighted KS Statistic: {col_metrics['ks_test_unweighted']:.4f}")
#         print(f"Weighted KS Statistic: {col_metrics['ks_test_weighted']:.4f}")

# def analyze_and_visualize_distributions(
#     full_dataset: pd.DataFrame,
#     training_subset: pd.DataFrame,
#     weights: pd.Series,
#     categorical_cols: List[str],
#     continuous_cols: List[str]
# ) -> Dict:
#     """
#     Analyzes and visualizes the distribution differences between full dataset and training subset,
#     before and after weighting.
#     """

#     # Plotting categorical distributions using heatmaps
#     for col in categorical_cols:
#         plot_categorical_distribution_heatmap(
#             full_dataset[col], training_subset[col], weights, col
#         )
#         plt.tight_layout()  # Ensure labels are visible
#         plt.show()

#     # Plotting empirical CDFs and histograms for continuous variables
#     for col in continuous_cols:
#         log_full_data = np.log1p(full_dataset[col])
#         log_training_data = np.log1p(training_subset[col])

#         # Plot empirical CDF
#         plt.figure(figsize=(10, 6))
#         plot_empirical_cdf(log_full_data, np.ones(len(full_dataset)), 'Full Dataset')
#         plot_empirical_cdf(log_training_data, np.ones(len(training_subset)), 'Training (Unweighted)')
#         plot_empirical_cdf(log_training_data, weights.values, 'Training (Weighted)')
#         plt.title(f'Empirical CDF of Log-{col}')
#         plt.legend()
#         plt.xlabel(f'Log-{col}')
#         plt.ylabel('Cumulative Probability')
#         plt.grid(True, alpha=0.3)
#         plt.tight_layout()
#         plt.show()

#         # Plot weighted histogram
#         plt.figure(figsize=(10, 6))
#         plot_weighted_histogram(log_full_data, np.ones(len(full_dataset)), 'Full Dataset')
#         plot_weighted_histogram(log_training_data, np.ones(len(training_subset)), 'Training (Unweighted)')
#         plot_weighted_histogram(log_training_data, weights.values, 'Training (Weighted)')
#         plt.title(f'Histogram of Log-{col}')
#         plt.legend()
#         plt.xlabel(f'Log-{col}')
#         plt.ylabel('Frequency')
#         plt.grid(True, alpha=0.3)
#         plt.tight_layout()
#         plt.show()

#     # Metrics Calculation
#     metrics = calculate_distribution_metrics(
#         full_dataset, training_subset, weights, categorical_cols, continuous_cols
#     )

#     # Display Results
#     display_results_table(metrics)

#     return metrics

# def add_population_bins(df, reference_df, n_bins=10):
#     df = df.copy()
#     # Calculate bin edges using the reference dataset
#     _, bin_edges = pd.qcut(reference_df['DEM: Population'], q=n_bins, retbins=True, duplicates='drop')
#     # Create bin labels
#     bin_labels = [f"{int(bin_edges[i])}-{int(bin_edges[i+1])}" for i in range(len(bin_edges)-1)]
#     # Apply these same bin edges to the input dataset
#     df['Population_Bin'] = pd.cut(df['DEM: Population'], bins=bin_edges, labels=bin_labels, include_lowest=True)
#     return df

# # Assuming df_cleaned and updated_df_train are already defined DataFrames
# # Create population bins using the full dataset's distribution as reference
# full_dataset_with_bins = add_population_bins(df_cleaned, df_cleaned)
# training_subset_with_bins = add_population_bins(updated_df_train, df_cleaned)  # Use same reference

# # Generate weights with adjustable aggressiveness
# weights = generate_weights_via_logistic_regression(
#     full_dataset=full_dataset_with_bins,
#     training_subset=training_subset_with_bins,
#     continuous_cols=['DEM: Population'],  # Can be empty list if no continuous columns
#     categorical_cols=['Population_Bin'],  # Can be empty list if no categorical columns
#     aggressiveness=2.5  # Adjust this value to make the fitting more or less aggressive
# )

# categorical_cols = ['Population_Bin']
# continuous_cols = ['DEM: Population']

# # Analyze and visualize
# distribution_metrics = analyze_and_visualize_distributions(
#     full_dataset_with_bins,
#     training_subset_with_bins,
#     weights,
#     categorical_cols,
#     continuous_cols
# )

# # Add weights to training subset
# updated_df_train['average_weight'] = weights


#### Save training data (before interactions or imputations):

This saves an intermediate version before removing identifiers and creating interactions.

In [34]:
updated_df_train['Location'] = updated_df_train['Location'].astype(str)

updated_df_train.to_parquet(folder_path_OUTPUT_VARIABLES + f"DF_Train_PreInteractions_PreImputations_{name_suffix}.parquet")
updated_df_train.to_csv(folder_path_OUTPUT_VARIABLES + f"DF_Train_PreInteractions_PreImputations_{name_suffix}.csv")
updated_df_train

<ipython-input-34-002c7c38e06f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_df_train['Location'] = updated_df_train['Location'].astype(str)


,Year,FIPS_2020,DF Name,ActivityProducedBy,ActivityConsumedBy,PointOfMeasurement,TemporalDataQuality,GeographicDataQuality,CompletenessDataQuality,DataReliability,...,IP: Federal general government (defense - GFGD),IP: Federal general government (nondefense - GFGN),IP: State and local government enterprises (GSLE),IP: State and local general government (GSLG),IP: Housing (531 real estate),IP: Other real estate (531 - not housing),Combo,count_reps,merge_predictors,original_index
211,2000,S06037,df431_0_0,Unspecified MSW,"Landfill,MRF","Landfill,MRF",1.0,1.0,1.0,5.0,...,0.009897,0.004027,0.010853,0.041523,0.009794,0.003516,None,NaN,both,211
1807,2000,S34001,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,None,NaN,both,1807
1808,2000,S34003,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,None,NaN,both,1808
1809,2000,S34005,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,None,NaN,both,1809
1810,2000,S34007,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,None,NaN,both,1810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72174,2022,S36047,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,None,NaN,both,72174
72181,2022,S36061,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,None,NaN,both,72181
72191,2022,S36081,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,None,NaN,both,72191
72193,2022,S36085,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,None,NaN,both,72193


# Imputations cleaning and variables interactions:

## Clean data imputations:

The following is to choose what variables to use in the model. An important part is choosing whether to use imputed values or not.

In [35]:
# Load data from imputations, generated in GeneratorImputation notebook.
df_generators_imputation = pd.read_csv(folder_path_OUTPUT_VARIABLES + 'df_imputations_filled_sub.csv')
df_generators_imputation.rename(columns={'FIPS': 'FIPS_2020'}, inplace=True)
df_generators_imputation[['ActivityProducedBy_Commercial_Imputed',	'ActivityProducedBy_Industrial_Imputed',	'ActivityProducedBy_Institutional_Imputed', 	'ActivityProducedBy_Residential_Imputed']].value_counts(dropna=False)

ActivityProducedBy_Commercial_Imputed  ActivityProducedBy_Industrial_Imputed  ActivityProducedBy_Institutional_Imputed  ActivityProducedBy_Residential_Imputed
1.0                                    1.0                                    0.0                                       1.0                                       1438
                                                                              1.0                                       1.0                                        534
                                       0.0                                    0.0                                       1.0                                        297
                                                                              1.0                                       1.0                                         99
0.0                                    0.0                                    0.0                                       1.0                                         27
1.0                                    0.0                                    0.0                                       0.0                                         14
                                                                              1.0                                       0.0                                          5
0.0                                    0.0                                    0.0                                       0.0                                          1
Name: count, dtype: int64

In [36]:
# df_generators_imputation_original=df_generators_imputation.copy()
# df_generators_imputation[['FIPS_2020', 'Year']].drop_duplicates()
#I want to tag the cases with duplicate id_variables FIPS_2020 and Year. i dont wnat to delete them, just tag them. I want to tag all ocurrences, even the first one
# df_generators_imputation['is_duplicated'] = df_generators_imputation.duplicated(subset=['FIPS_2020', 'Year'], keep=False)
# MY previous methods did not tag all ocurrences, i need to make it tag all ocurrencces of repeated cases.
df_generators_imputation['is_duplicated'] = df_generators_imputation.duplicated(subset=['FIPS_2020', 'Year'], keep=False)

# sort by id variables:
df_generators_imputation = df_generators_imputation.sort_values(by=['FIPS_2020', 'Year'])
df_generators_imputation[df_generators_imputation['is_duplicated']]

,FIPS_2020,Year,ActivityProducedBy_Commercial_Imputed,ActivityProducedBy_Industrial_Imputed,ActivityProducedBy_Institutional_Imputed,ActivityProducedBy_Residential_Imputed,is_duplicated
2020,S08013,2018,0.0,0.0,0.0,1.0,True
2158,S08013,2018,1.0,0.0,0.0,0.0,True
2022,S15003,2017,0.0,0.0,0.0,1.0,True
2159,S15003,2017,1.0,0.0,0.0,0.0,True
103,S17111,2008,0.0,0.0,0.0,1.0,True
...,...,...,...,...,...,...,...
2167,S42045,2010,1.0,0.0,0.0,0.0,True
1964,S42045,2011,0.0,0.0,0.0,1.0,True
2168,S42045,2011,1.0,0.0,0.0,0.0,True
1965,S42045,2012,0.0,0.0,0.0,1.0,True


In [37]:
df_train_generators_pre_imput = updated_df_train.copy()
print(pd.crosstab(df_train_generators_pre_imput['ActivityProducedBy_Residential'].isna(), df_train_generators_pre_imput['ActivityProducedBy_Unspecified MSW'], dropna=False))
df_train_generators_pre_imput[['ActivityProducedBy_Unspecified MSW', 'ActivityProducedBy_Residential', 'ActivityProducedBy_Institutional', 'ActivityProducedBy_Commercial','ActivityProducedBy_Industrial']].value_counts(dropna=False)

ActivityProducedBy_Unspecified MSW  0.0   1.0
ActivityProducedBy_Residential               
False                               779  1627


ActivityProducedBy_Unspecified MSW  ActivityProducedBy_Residential  ActivityProducedBy_Institutional  ActivityProducedBy_Commercial  ActivityProducedBy_Industrial
1.0                                 0.0                             0.0                               0.0                            0.0                              1367
0.0                                 1.0                             1.0                               1.0                            1.0                               506
1.0                                 0.0                             0.0                               0.0                            1.0                               260
0.0                                 1.0                             0.0                               1.0                            0.0                               154
                                                                    1.0                               1.0                            0.0                                52
                                                                    0.0                               1.0                            1.0                                31
                                                                                                      0.0                            0.0                                17
                                    0.0                             0.0                               1.0                            0.0                                14
                                                                    1.0                               1.0                            0.0                                 5
Name: count, dtype: int64

In [38]:
df_train_generators = pd.merge(df_train_generators_pre_imput, df_generators_imputation, on=['Year', 'FIPS_2020'], how='outer', indicator='_merge_imputations')
print(df_train_generators._merge_imputations.value_counts())
df_train_generators = df_train_generators[df_train_generators._merge_imputations == 'both']
df_train_generators

_merge_imputations
both          2470
right_only       9
left_only        0
Name: count, dtype: int64


,Year,FIPS_2020,DF Name,ActivityProducedBy,ActivityConsumedBy,PointOfMeasurement,TemporalDataQuality,GeographicDataQuality,CompletenessDataQuality,DataReliability,...,Combo,count_reps,merge_predictors,original_index,ActivityProducedBy_Commercial_Imputed,ActivityProducedBy_Industrial_Imputed,ActivityProducedBy_Institutional_Imputed,ActivityProducedBy_Residential_Imputed,is_duplicated,_merge_imputations
1,2000,S06037,df431_0_0,Unspecified MSW,"Landfill,MRF","Landfill,MRF",1.0,1.0,1.0,5.0,...,None,NaN,both,211.0,1.0,0.0,1.0,1.0,False,both
2,2000,S34001,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,None,NaN,both,1807.0,1.0,1.0,0.0,1.0,False,both
3,2000,S34003,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,None,NaN,both,1808.0,1.0,1.0,0.0,1.0,False,both
4,2000,S34005,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,None,NaN,both,1809.0,1.0,1.0,0.0,1.0,False,both
5,2000,S34007,df406_0_0,Unspecified MSW,"Landfill,Recycling",Unspecified Waste Facility,1.0,5.0,5.0,1.0,...,None,NaN,both,1810.0,1.0,1.0,0.0,1.0,False,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,None,NaN,both,72174.0,1.0,0.0,0.0,1.0,False,both
2469,2022,S36061,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,None,NaN,both,72181.0,1.0,0.0,0.0,1.0,False,both
2470,2022,S36081,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,None,NaN,both,72191.0,1.0,0.0,0.0,1.0,False,both
2471,2022,S36085,df432_0_0,Unspecified MSW,"Landfill,Recycling",Curbside Collection,1.0,1.0,1.0,5.0,...,None,NaN,both,72193.0,1.0,0.0,0.0,1.0,False,both


In [39]:
# Keep either imputed or original values depending on "keep_imputations option".
print(pd.crosstab(df_train_generators['ActivityProducedBy_Residential_Imputed'],
                  df_train_generators['ActivityProducedBy_Residential'], dropna=False))
print(pd.crosstab(df_train_generators['ActivityProducedBy_Institutional_Imputed'],
                  df_train_generators['ActivityProducedBy_Institutional'], dropna=False))
print(pd.crosstab(df_train_generators['ActivityProducedBy_Commercial_Imputed'],
                  df_train_generators['ActivityProducedBy_Commercial'], dropna=False))
print(pd.crosstab(df_train_generators['ActivityProducedBy_Industrial_Imputed'],
                  df_train_generators['ActivityProducedBy_Industrial'], dropna=False))

ActivityProducedBy_Residential           0.0  1.0
ActivityProducedBy_Residential_Imputed           
0.0                                       25   12
1.0                                     1650  783
ActivityProducedBy_Institutional           0.0  1.0
ActivityProducedBy_Institutional_Imputed           
0.0                                       1795   19
1.0                                         93  563
ActivityProducedBy_Commercial           0.0  1.0
ActivityProducedBy_Commercial_Imputed           
0.0                                      28   12
1.0                                    1640  790
ActivityProducedBy_Industrial           0.0  1.0
ActivityProducedBy_Industrial_Imputed           
0.0                                     475    9
1.0                                    1189  797


In [40]:
print(df_train_generators[[ 'ActivityProducedBy_Unspecified MSW', 'ActivityProducedBy_Residential', 'ActivityProducedBy_Residential_Imputed']].value_counts(dropna=False))
print(df_train_generators[[ 'ActivityProducedBy_Unspecified MSW', 'ActivityProducedBy_Institutional', 'ActivityProducedBy_Institutional_Imputed']].value_counts(dropna=False))
print(df_train_generators[[ 'ActivityProducedBy_Unspecified MSW', 'ActivityProducedBy_Commercial', 'ActivityProducedBy_Commercial_Imputed']].value_counts(dropna=False))
print(df_train_generators[[ 'ActivityProducedBy_Unspecified MSW', 'ActivityProducedBy_Industrial', 'ActivityProducedBy_Industrial_Imputed']].value_counts(dropna=False))

ActivityProducedBy_Unspecified MSW  ActivityProducedBy_Residential  ActivityProducedBy_Residential_Imputed
1.0                                 0.0                             1.0                                       1633
0.0                                 1.0                             1.0                                        783
                                    0.0                             0.0                                         19
                                                                    1.0                                         17
                                    1.0                             0.0                                         12
1.0                                 0.0                             0.0                                          6
Name: count, dtype: int64
ActivityProducedBy_Unspecified MSW  ActivityProducedBy_Institutional  ActivityProducedBy_Institutional_Imputed
1.0                                 0.0                           

In [41]:
imputed_list = ['ActivityProducedBy_Residential', 'ActivityProducedBy_Institutional', 'ActivityProducedBy_Commercial','ActivityProducedBy_Industrial']

if small_sample:
    df_train_generators = df_train_generators[df_train_generators['ActivityProducedBy_Unspecified MSW'] != 1]
    df_train_generators = df_train_generators[df_train_generators['ActivityProducedBy_Residential'].notna()]

if keep_imputations:
    for col in imputed_list:
        df_train_generators[col]= np.where(df_train_generators[col]==1, df_train_generators[col], df_train_generators[col + '_Imputed']) # keep imputed values unless original value was a "1" (i.e., a known generator included in the sample)
        df_train_generators.drop(col + '_Imputed', axis=1, inplace=True)
else:
    for col in imputed_list:
        df_train_generators.drop([col + '_Imputed'], axis=1, inplace=True)
        df_train_generators[col] = df_train_generators[col].fillna(0)

df_train_generators[['ActivityProducedBy_Residential', 'ActivityProducedBy_Institutional', 'ActivityProducedBy_Commercial','ActivityProducedBy_Industrial', 'ActivityProducedBy_Unspecified MSW']]

,ActivityProducedBy_Residential,ActivityProducedBy_Institutional,ActivityProducedBy_Commercial,ActivityProducedBy_Industrial,ActivityProducedBy_Unspecified MSW
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
2468,0.0,0.0,0.0,0.0,1.0
2469,0.0,0.0,0.0,0.0,1.0
2470,0.0,0.0,0.0,0.0,1.0
2471,0.0,0.0,0.0,0.0,1.0


## Remove identifiers from dataset:

In [42]:
list(df_train_generators.columns)

['Year',
 'FIPS_2020',
 'DF Name',
 'ActivityProducedBy',
 'ActivityConsumedBy',
 'PointOfMeasurement',
 'TemporalDataQuality',
 'GeographicDataQuality',
 'CompletenessDataQuality',
 'DataReliability',
 'FlowName',
 'Waste Generation Units',
 'Recycling Included',
 'Disposal Included',
 'Yard Waste Included',
 'Food Waste Included',
 'Waste Generation Amount',
 'Location',
 'ActivityConsumedBy_Compost',
 'ActivityConsumedBy_Incinerator',
 'ActivityConsumedBy_Landfill',
 'ActivityConsumedBy_MRF',
 'ActivityConsumedBy_Recycling',
 'ActivityConsumedBy_Unknown',
 'ActivityProducedBy_C&D',
 'ActivityProducedBy_Commercial',
 'ActivityProducedBy_Industrial',
 'ActivityProducedBy_Institutional',
 'ActivityProducedBy_Landscape',
 'ActivityProducedBy_Residential',
 'ActivityProducedBy_Unspecified MSW',
 'PointOfMeasurement_Compost',
 'PointOfMeasurement_Composting',
 'PointOfMeasurement_Curbside Collection',
 'PointOfMeasurement_Incinerator/Combustion',
 'PointOfMeasurement_Landfill',
 'PointOfM

In [43]:
df_train_generators = df_train_generators.drop(['Land Area (sq miles)',
                          'stateFIPS',
                          'Combo',
                          'count_reps',
                          'merge_predictors',
                          'original_index',
                          '_merge_imputations',
                          'State',
                          'FIPS_before_map',
                          'County',
                          'County_2020',
                          'Location',
                          'DF Name'], axis=1, inplace=False)

In [44]:
df_train_generators.to_parquet(folder_path_OUTPUT_VARIABLES + f"DF_Train_PreInteractions_{name_suffix}_WITH_IDENTIFIERS.parquet")
print("File saved to: " + folder_path_OUTPUT_VARIABLES + f"DF_Train_PreInteractions_{name_suffix}_WITH_IDENTIFIERS.parquet")

File saved to: /content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/DF_Train_PreInteractions_KeepImp0_GenInter0_SmallSample0_WITH_IDENTIFIERS.parquet


In [45]:
df_TRAIN = df_train_generators.copy()

id_variables = ['Year', 'FIPS_2020']
transformed_variables = ['ActivityProducedBy', 'ActivityConsumedBy', 'FlowName', 'Waste Generation Units', 'PointOfMeasurement', 'DEM: Median Household Income']
data_quality_variable = ['DataReliability', 'CompletenessDataQuality', 'TemporalDataQuality', 'GeographicDataQuality']

df_TRAIN = df_TRAIN.drop(data_quality_variable + transformed_variables, axis=1, inplace=False)

print(df_TRAIN.shape)
list(df_TRAIN.columns)


(2470, 268)


['Year',
 'FIPS_2020',
 'Recycling Included',
 'Disposal Included',
 'Yard Waste Included',
 'Food Waste Included',
 'Waste Generation Amount',
 'ActivityConsumedBy_Compost',
 'ActivityConsumedBy_Incinerator',
 'ActivityConsumedBy_Landfill',
 'ActivityConsumedBy_MRF',
 'ActivityConsumedBy_Recycling',
 'ActivityConsumedBy_Unknown',
 'ActivityProducedBy_C&D',
 'ActivityProducedBy_Commercial',
 'ActivityProducedBy_Industrial',
 'ActivityProducedBy_Institutional',
 'ActivityProducedBy_Landscape',
 'ActivityProducedBy_Residential',
 'ActivityProducedBy_Unspecified MSW',
 'PointOfMeasurement_Compost',
 'PointOfMeasurement_Composting',
 'PointOfMeasurement_Curbside Collection',
 'PointOfMeasurement_Incinerator/Combustion',
 'PointOfMeasurement_Landfill',
 'PointOfMeasurement_MRF',
 'PointOfMeasurement_Material Recovery Facility',
 'PointOfMeasurement_Recycling',
 'PointOfMeasurement_Self-Haul',
 'PointOfMeasurement_Transfer Station',
 'PointOfMeasurement_Unspecified Waste Facility',
 'PointOf

In [46]:
df_TRAIN.isna().sum().sort_values(ascending=False)

,0
Year,0
PCE: Gasoline and other energy goods,0
POL: Subsidize user fee,0
POL: Promote user fee,0
POL: Landfill tax,0
...,...
"CBP: Agriculture, Forestry, Fishing and Hunting - (11 excl. 111, 112)",0
"CBP: Web Search Portals, Libraries, Archives, and Other Information Services (519)",0
CBP: Air Transportation (481),0
CBP: Administrative and Support and Waste Management and Remediation Services (56),0


## Define variables interactions (generator-specific variables).

This part will generate interaction terms between generators and independent variables to convert them into generator-specific variables (i.e., the only contribute to the predicted output for those generators).

In [47]:
var_Residential = ["DEM: Population", "DEM: Median Household Income", "DEM: Proportion Below Poverty Line", "DEM: Unemployment Rate", "DEM: less than $10,000", "DEM: $10,000 - $14,999", "DEM: $15,000 - $24,999", "DEM: $25,000 - $34,999", "DEM: $35,000 - $49,999", "DEM: $50,000 - $74,999", "DEM: $75,000 - $99,999", "DEM: $100,000 - $149,999", "DEM: $150,000 - $199,999", "DEM: $200,000 and over", "DEM: Owner-Occupied Housing", "DEM: Under 5 years", "DEM: 5-9 years", "DEM: 10-14 years", "DEM: 15-19 years", "DEM: 20-24 years", "DEM: 25-34 years", "DEM: 35-44 years", "DEM: 45-54 years", "DEM: 55-59 years", "DEM: 60-64 years", "DEM: 65-74 years", "DEM: 75-84 years", "DEM: 85+ years", "DEM: Bachelors Degree", "DEM: High School Degree", "INF: # MSW Landfills", "INF: # TSs", "INF: # MRF", "POL: Require curbside recycling", "POL: Required recycling goal", "PCE: Housing and utilities", "PCE: Motor vehicles and parts", "PCE: Other durable goods", "PCE: Other nondurable goods", "PCE: Other services", "PCE: Recreation services", "PCE: Recreational goods and vehicles", "PCE: Transportation services", "PCE: Durable goods", "PCE: Final consumption expenditures of nonprofit institutions serving households (NPISHs)", "PCE: Household consumption expenditures (for services)", "PCE: Nondurable goods", "PCE: Goods", "PCE: Services", "PCE: Personal consumption expenditures", "PCE: Sales of goods and services by nonprofit institutions", "PCE: Health care", "POL: Require user fee", "PCE: Gross output of nonprofit institutions", "POL: Subsidize user fee", "POL: Promote user fee", "POL: Landfill tax", "POL: Tipping Fee", "POL: Bottle Bill Price", "POL: Plastic Bag Ban", "POL: Littering is Criminal", "POL: Littering Fine", "POL: Littering Penalties", "PCE: Clothing and footwear", "PCE: Financial services and insurance", "PCE: Food and beverages purchased for off-premises consumption", "PCE: Furnishings and durable household equipment", "PCE: Gasoline and other energy goods"]
var_Commercial = ["CBP: Agriculture, Forestry, Fishing and Hunting (11)", "CBP: Forestry and Logging (113)", "CBP: Fishing, Hunting and Trapping (114)", "CBP: Support Activities for Crop Production (115)", "CBP: Mining, Quarrying, and Oil and Gas Extraction (21)", "CBP: Oil and Gas Extraction (211)", "CBP: Mining (except Oil and Gas) (212)", "CBP: Support Activities for Mining (213)", "CBP: Utilities (22)", "CBP: Utilities (221)", "CBP: Construction (23)", "CBP: Construction of Buildings (236)", "CBP: Heavy and Civil Engineering Construction (237)", "CBP: Specialty Trade Contractors (238)", "CBP: Food Manufacturing (311)", "CBP: Beverage and Tobacco Product Manufacturing (312)", "CBP: Textile Mills (313)", "CBP: Manufacturing (31-33 )", "CBP: Textile Product Mills (314)", "CBP: Apparel Manufacturing (315)", "CBP: Leather and Allied Product Manufacturing (316)", "CBP: Wood Product Manufacturing (321)", "CBP: Paper Manufacturing (322)", "CBP: Printing and Related Support Activities (323)", "CBP: Petroleum and Coal Products Manufacturing (324)", "CBP: Chemical Manufacturing (325)", "CBP: Plastics and Rubber Products Manufacturing (326)", "CBP: Nonmetallic Mineral Product Manufacturing (327)", "CBP: Primary Metal Manufacturing (331)", "CBP: Fabricated Metal Product Manufacturing (332)", "CBP: Machinery Manufacturing (333)", "CBP: Computer and Electronic Product Manufacturing (334)", "CBP: Electrical Equipment, Appliance, and Component Manufacturing (335)", "CBP: Transportation Equipment Manufacturing (336)", "CBP: Furniture and Related Product Manufacturing (337)", "CBP: Miscellaneous Manufacturing (339)", "CBP: Wholesale Trade (42)", "CBP: Merchant Wholesalers, Durable Goods (423)", "CBP: Merchant Wholesalers, Nondurable Goods (424)", "CBP: Wholesale Trade Agents and Brokers (425)", "CBP: Motor Vehicle and Parts Dealers (441)", "CBP: Building Material and Garden Equipment and Supplies Dealers (444)", "CBP: Retail Trade (44-45 )", "CBP: Food and Beverage Retailers (445)", "CBP: Air Transportation (481)", "CBP: Water Transportation (483)", "CBP: Truck Transportation (484)", "CBP: Transportation and Warehousing (48-49 )", "CBP: Transit and Ground Passenger Transportation (485)", "CBP: Pipeline Transportation (486)", "CBP: Scenic and Sightseeing Transportation (487)", "CBP: Support Activities for Transportation (488)", "CBP: Couriers and Messengers (492)", "CBP: Warehousing and Storage (493)", "CBP: Information (51)", "CBP: Motion Picture and Sound Recording Industries (512)", "CBP: Telecommunications (517)", "CBP: Computing Infrastructure Providers, Data Processing, Web Hosting (518)", "CBP: Web Search Portals, Libraries, Archives, and Other Information Services (519)", "CBP: Finance and Insurance (52)", "CBP: Monetary Authorities-Central Bank (521)", "CBP: Credit Intermediation and Related Activities (522)", "CBP: Securities, Commodity Contracts, and Other Financial Investments and Related Activities (523)", "CBP: Insurance Carriers and Related Activities (524)", "CBP: Funds, Trusts, and Other Financial Vehicles (525)", "CBP: Real Estate and Rental and Leasing (53)", "CBP: Real Estate (531)", "CBP: Rental and Leasing Services (532)", "CBP: Lessors of Nonfinancial Intangible Assets (except Copyrighted Works) (533)", "CBP: Professional, Scientific, and Technical Services (54)", "CBP: Professional, Scientific, and Technical Services (541)", "CBP: Management of Companies and Enterprises (55)", "CBP: Management of Companies and Enterprises (551)", "CBP: Administrative and Support and Waste Management and Remediation Services (56)", "CBP: Administrative and Support Services (561)", "CBP: Waste Management and Remediation Services (562)", "CBP: Educational Services (61)", "CBP: Educational Services (611)", "CBP: Health Care and Social Assistance (62)", "CBP: Ambulatory Health Care Services (621)", "CBP: Hospitals (622)", "CBP: Nursing and Residential Care Facilities (623)", "CBP: Social Assistance (624)", "CBP: Arts, Entertainment, and Recreation (71)", "CBP: Performing Arts, Spectator Sports, and Related Industries (711)", "CBP: Museums, Historical Sites, and Similar Institutions (712)", "CBP: Amusement, Gambling, and Recreation Industries (713)", "CBP: Accommodation and Food Services (72)", "CBP: Accommodation (721)", "CBP: Food Services and Drinking Places (722)", "CBP: Other Services (except Public Administration) (81)", "CBP: Repair and Maintenance (811)", "CBP: Personal and Laundry Services (812)", "CBP: Religious, Grantmaking, Civic, Professional, and Similar Organizations (813)", "INF: # MSW Landfills", "INF: # TSs", "INF: # MRF", "Total Nursery in Protection Production Area (sq. ft.)", "Total Nursery in Open Production Area (acres)", "Total Irrigated Vegetable Area (acres)", "POL: Required recycling goal", "POL: Require user fee", "POL: Subsidize user fee", "POL: Promote user fee", "POL: Landfill tax", "POL: Tipping Fee", "POL: Bottle Bill Price", "POL: Plastic Bag Ban", "POL: Littering is Criminal", "POL: Littering Fine", "POL: Littering Penalties", "IP: Textile mills and textile product mills", "IP: Motor vehicles, bodies and trailers, and parts", "IP: Furniture and related products", "IP: Miscellaneous manufacturing", "IP: Wholesale trade", "IP: Food and beverage stores", "IP: Air transportation", "IP: Water transportation", "IP: Truck transportation", "IP: Transit and ground passenger transportation", "IP: Pipeline transportation", "IP: Other transportation and support activities", "IP: Warehousing and storage", "IP: Motion picture and sound recording industries", "IP: Federal Reserve banks, credit intermediation, and related activities", "IP: Other transportation equipment", "IP: Electrical equipment, appliances, and components", "IP: Insurance carriers and related activities", "IP: Computer and electronic products", "IP: Forestry, fishing, and related activities", "IP: Oil and gas extraction", "IP: Mining, except oil and gas", "IP: Support activities for mining", "IP: Utilities", "IP: Construction", "IP: Food and beverage and tobacco products", "IP: Apparel and leather and allied products", "IP: Wood products", "IP: Paper products", "IP: Printing and related support activities", "IP: Petroleum and coal products", "IP: Chemical products", "IP: Plastics and rubber products", "IP: Nonmetallic mineral products", "IP: Primary metals", "IP: Fabricated metal products", "IP: Machinery", "IP: Securities, commodity contracts, and investments", "IP: Motor vehicle and parts dealers", "IP: Funds, trusts, and other financial vehicles", "IP: Rental and leasing services and lessors of intangible assets", "IP: Hospitals", "IP: Legal services", "IP: Miscellaneous professional, scientific, and technical services", "IP: Computer systems design and related services", "IP: Management of companies and enterprises", "IP: Administrative and support services", "IP: Waste management and remediation services", "IP: Educational services", "IP: Ambulatory health care services", "IP: Nursing and residential care facilities", "IP: Accommodation", "IP: Food services and drinking places", "IP: Other services, except government", "IP: Amusements, gambling, and recreation industries", "IP: Performing arts, spectator sports, museums, and related activities", "IP: Social assistance", "IP: Federal government enterprises (GFE)", "IP: Federal general government (defense - GFGD)", "IP: Federal general government (nondefense - GFGN)", "IP: State and local government enterprises (GSLE)", "IP: State and local general government (GSLG)", "IP: Housing (531 real estate)", "IP: Other real estate (531 - not housing)"]
var_Industrial = ["CBP: Agriculture, Forestry, Fishing and Hunting (11)", "CBP: Forestry and Logging (113)", "CBP: Fishing, Hunting and Trapping (114)", "CBP: Support Activities for Crop Production (115)", "CBP: Mining, Quarrying, and Oil and Gas Extraction (21)", "CBP: Oil and Gas Extraction (211)", "CBP: Mining (except Oil and Gas) (212)", "CBP: Support Activities for Mining (213)", "CBP: Utilities (22)", "CBP: Utilities (221)", "CBP: Food Manufacturing (311)", "CBP: Beverage and Tobacco Product Manufacturing (312)", "CBP: Textile Mills (313)", "CBP: Manufacturing (31-33 )", "CBP: Textile Product Mills (314)", "CBP: Apparel Manufacturing (315)", "CBP: Leather and Allied Product Manufacturing (316)", "CBP: Wood Product Manufacturing (321)", "CBP: Paper Manufacturing (322)", "CBP: Printing and Related Support Activities (323)", "CBP: Petroleum and Coal Products Manufacturing (324)", "CBP: Chemical Manufacturing (325)", "CBP: Plastics and Rubber Products Manufacturing (326)", "CBP: Nonmetallic Mineral Product Manufacturing (327)", "CBP: Primary Metal Manufacturing (331)", "CBP: Fabricated Metal Product Manufacturing (332)", "CBP: Machinery Manufacturing (333)", "CBP: Computer and Electronic Product Manufacturing (334)", "CBP: Electrical Equipment, Appliance, and Component Manufacturing (335)", "CBP: Transportation Equipment Manufacturing (336)", "CBP: Furniture and Related Product Manufacturing (337)", "CBP: Miscellaneous Manufacturing (339)", "CBP: Information (51)", "CBP: Motion Picture and Sound Recording Industries (512)", "CBP: Telecommunications (517)", "CBP: Computing Infrastructure Providers, Data Processing, Web Hosting (518)", "CBP: Web Search Portals, Libraries, Archives, and Other Information Services (519)", "CBP: Finance and Insurance (52)", "CBP: Monetary Authorities-Central Bank (521)", "CBP: Credit Intermediation and Related Activities (522)", "CBP: Securities, Commodity Contracts, and Other Financial Investments and Related Activities (523)", "CBP: Insurance Carriers and Related Activities (524)", "CBP: Funds, Trusts, and Other Financial Vehicles (525)", "CBP: Real Estate and Rental and Leasing (53)", "CBP: Real Estate (531)", "CBP: Rental and Leasing Services (532)", "CBP: Lessors of Nonfinancial Intangible Assets (except Copyrighted Works) (533)", "CBP: Professional, Scientific, and Technical Services (54)", "CBP: Professional, Scientific, and Technical Services (541)", "CBP: Management of Companies and Enterprises (55)", "CBP: Management of Companies and Enterprises (551)", "CBP: Administrative and Support and Waste Management and Remediation Services (56)", "CBP: Administrative and Support Services (561)", "CBP: Waste Management and Remediation Services (562)", "INF: # MSW Landfills", "INF: # TSs", "INF: # MRF", "POL: Subsidize user fee", "POL: Promote user fee", "POL: Landfill tax", "POL: Tipping Fee", "IP: Textile mills and textile product mills", "IP: Motor vehicles, bodies and trailers, and parts", "IP: Furniture and related products", "IP: Miscellaneous manufacturing", "IP: Wholesale trade", "IP: Food and beverage stores", "IP: Air transportation", "IP: Water transportation", "IP: Truck transportation", "IP: Transit and ground passenger transportation", "IP: Pipeline transportation", "IP: Other transportation and support activities", "IP: Warehousing and storage", "IP: Motion picture and sound recording industries", "IP: Federal Reserve banks, credit intermediation, and related activities", "IP: Other transportation equipment", "IP: Electrical equipment, appliances, and components", "IP: Insurance carriers and related activities", "IP: Computer and electronic products", "IP: Forestry, fishing, and related activities", "IP: Oil and gas extraction", "IP: Mining, except oil and gas", "IP: Support activities for mining", "IP: Utilities", "IP: Construction", "IP: Food and beverage and tobacco products", "IP: Apparel and leather and allied products", "IP: Wood products", "IP: Paper products", "IP: Printing and related support activities", "IP: Petroleum and coal products", "IP: Chemical products", "IP: Plastics and rubber products", "IP: Nonmetallic mineral products", "IP: Primary metals", "IP: Fabricated metal products", "IP: Machinery", "IP: Securities, commodity contracts, and investments", "IP: Motor vehicle and parts dealers", "IP: Funds, trusts, and other financial vehicles", "IP: Rental and leasing services and lessors of intangible assets", "IP: Hospitals", "IP: Legal services", "IP: Miscellaneous professional, scientific, and technical services", "IP: Computer systems design and related services", "IP: Management of companies and enterprises", "IP: Administrative and support services", "IP: Waste management and remediation services", "IP: Educational services", "IP: Ambulatory health care services", "IP: Nursing and residential care facilities", "IP: Accommodation", "IP: Food services and drinking places", "IP: Other services, except government", "IP: Amusements, gambling, and recreation industries", "IP: Performing arts, spectator sports, museums, and related activities", "IP: Social assistance", "IP: Federal government enterprises (GFE)", "IP: Federal general government (defense - GFGD)", "IP: Federal general government (nondefense - GFGN)", "IP: State and local government enterprises (GSLE)", "IP: State and local general government (GSLG)", "IP: Housing (531 real estate)", "IP: Other real estate (531 - not housing)"]
var_Institutional = ["CBP: Educational Services (61)", "CBP: Educational Services (611)", "CBP: Health Care and Social Assistance (62)", "CBP: Ambulatory Health Care Services (621)", "CBP: Hospitals (622)", "CBP: Nursing and Residential Care Facilities (623)", "CBP: Social Assistance (624)", "INF: # MSW Landfills", "INF: # TSs", "INF: # MRF", "Prop. of population that attends church", "Number of churches", "POL: Required recycling goal", "POL: Require user fee", "POL: Subsidize user fee", "POL: Promote user fee", "POL: Landfill tax", "POL: Tipping Fee", "POL: Bottle Bill Price", "POL: Plastic Bag Ban", "POL: Littering is Criminal", "POL: Littering Fine", "POL: Littering Penalties", "University enrollment"]

# XXX CHECK ADD VARIABLES AFFECTING ALL GROUPS AS INTERACTIONS?

with open(folder_path_OUTPUT_VARIABLES + f"var_Residential_{name_suffix}.pkl", "wb") as file:
    pickle.dump(var_Residential, file)
with open(folder_path_OUTPUT_VARIABLES + f"var_Commercial_{name_suffix}.pkl", "wb") as file:
    pickle.dump(var_Commercial, file)
with open(folder_path_OUTPUT_VARIABLES + f"var_Industrial_{name_suffix}.pkl", "wb") as file:
    pickle.dump(var_Industrial, file)
with open(folder_path_OUTPUT_VARIABLES + f"var_Institutional_{name_suffix}.pkl", "wb") as file:
    pickle.dump(var_Institutional, file)

In [48]:
all_variables_list = list(set(var_Residential + var_Commercial + var_Institutional + var_Industrial).intersection(df_TRAIN.columns))

if generate_interactions_variables:
    # This is only temporary to include interactions with generator types:
    X_numerical = df_TRAIN.select_dtypes(include=['number'])
    X_categorical = df_TRAIN.select_dtypes(include=['object', 'category', 'bool'])

    def generate_interactions(X, var_Residential, var_Commercial, var_Institutional, var_Industrial, X_numerical, X_categorical):
        all_variables_list = list(set(var_Residential + var_Commercial + var_Institutional + var_Industrial).intersection(X.columns))
        X = X.copy()
        for var in X_numerical + X_categorical:
            if var in set(var_Residential).intersection(X.columns):
                generator = 'ActivityProducedBy_Residential'
                X[var + " x " + generator] = np.where(X[generator] == 1, X[var], 0)
            if var in set(var_Commercial).intersection(X.columns):
                generator = 'ActivityProducedBy_Commercial'
                X[var + " x " + generator] = np.where(X[generator] == 1, X[var], 0)
            if var in set(var_Institutional).intersection(X.columns):
                generator = 'ActivityProducedBy_Institutional'
                X[var + " x " + generator] = np.where(X[generator] == 1, X[var], 0)
            if var in set(var_Industrial).intersection(X.columns):
                generator = 'ActivityProducedBy_Industrial'
                X[var + " x " + generator] = np.where(X[generator] == 1, X[var], 0)

        X.drop(columns=all_variables_list, inplace=True, axis=1)
        return X

    df_TRAIN_interactions = generate_interactions(df_TRAIN, var_Residential, var_Commercial, var_Institutional, var_Industrial, X_numerical, X_categorical)
    df_TRAIN_interactions = df_TRAIN_interactions.copy()
else:
    df_TRAIN_interactions = df_TRAIN.copy()

In [49]:
df_TRAIN_interactions

,Year,FIPS_2020,Recycling Included,Disposal Included,Yard Waste Included,Food Waste Included,Waste Generation Amount,ActivityConsumedBy_Compost,ActivityConsumedBy_Incinerator,ActivityConsumedBy_Landfill,...,IP: Food services and drinking places,"IP: Other services, except government",IP: Federal government enterprises (GFE),IP: Federal general government (defense - GFGD),IP: Federal general government (nondefense - GFGN),IP: State and local government enterprises (GSLE),IP: State and local general government (GSLG),IP: Housing (531 real estate),IP: Other real estate (531 - not housing),is_duplicated
1,2000,S06037,0.0,1.0,0.0,0.0,11384354.0,0.0,0.0,1.0,...,0.009808,0.010016,0.002154,0.009897,0.004027,0.010853,0.041523,0.009794,0.003516,False
2,2000,S34001,1.0,1.0,0.0,0.0,329432.0,0.0,0.0,1.0,...,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,False
3,2000,S34003,1.0,1.0,0.0,0.0,1040243.0,0.0,0.0,1.0,...,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,False
4,2000,S34005,1.0,1.0,0.0,0.0,469772.0,0.0,0.0,1.0,...,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,False
5,2000,S34007,1.0,1.0,0.0,0.0,493305.0,0.0,0.0,1.0,...,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.0,1.0,0.0,1.0,831039.0,0.0,0.0,1.0,...,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,False
2469,2022,S36061,0.0,1.0,0.0,1.0,395511.3,0.0,0.0,1.0,...,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,False
2470,2022,S36081,0.0,1.0,0.0,1.0,711611.3,0.0,0.0,1.0,...,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,False
2471,2022,S36085,0.0,1.0,0.0,1.0,182057.8,0.0,0.0,1.0,...,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,False


## Save preprocessed data:

In [50]:

df_TRAIN_interactions.to_parquet(folder_path_OUTPUT_VARIABLES + f"DF_Train_PreProcessed_{name_suffix}.parquet")
df_TRAIN_interactions.to_csv(folder_path_OUTPUT_VARIABLES + f"DF_Train_PreProcessed_{name_suffix}.csv")

print("Data saved to folder path:", folder_path_OUTPUT_VARIABLES)
print("File name:", f"DF_Train_PreProcessed_{name_suffix}.parquet")
df_TRAIN_interactions

Data saved to folder path: /content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/
File name: DF_Train_PreProcessed_KeepImp0_GenInter0_SmallSample0.parquet


,Year,FIPS_2020,Recycling Included,Disposal Included,Yard Waste Included,Food Waste Included,Waste Generation Amount,ActivityConsumedBy_Compost,ActivityConsumedBy_Incinerator,ActivityConsumedBy_Landfill,...,IP: Food services and drinking places,"IP: Other services, except government",IP: Federal government enterprises (GFE),IP: Federal general government (defense - GFGD),IP: Federal general government (nondefense - GFGN),IP: State and local government enterprises (GSLE),IP: State and local general government (GSLG),IP: Housing (531 real estate),IP: Other real estate (531 - not housing),is_duplicated
1,2000,S06037,0.0,1.0,0.0,0.0,11384354.0,0.0,0.0,1.0,...,0.009808,0.010016,0.002154,0.009897,0.004027,0.010853,0.041523,0.009794,0.003516,False
2,2000,S34001,1.0,1.0,0.0,0.0,329432.0,0.0,0.0,1.0,...,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,False
3,2000,S34003,1.0,1.0,0.0,0.0,1040243.0,0.0,0.0,1.0,...,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,False
4,2000,S34005,1.0,1.0,0.0,0.0,469772.0,0.0,0.0,1.0,...,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,False
5,2000,S34007,1.0,1.0,0.0,0.0,493305.0,0.0,0.0,1.0,...,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.0,1.0,0.0,1.0,831039.0,0.0,0.0,1.0,...,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,False
2469,2022,S36061,0.0,1.0,0.0,1.0,395511.3,0.0,0.0,1.0,...,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,False
2470,2022,S36081,0.0,1.0,0.0,1.0,711611.3,0.0,0.0,1.0,...,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,False
2471,2022,S36085,0.0,1.0,0.0,1.0,182057.8,0.0,0.0,1.0,...,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240,False


# Scaling/standardization:

Define model data:

In [51]:
df_ReadyToTrain = df_TRAIN_interactions.copy()

target_col = 'Waste Generation Amount'

df_ReadyToTrain['average_weight'] = 1
sample_weights = df_ReadyToTrain['average_weight']

X = df_ReadyToTrain.drop(columns=[target_col, 'average_weight'])
y = df_ReadyToTrain[id_variables + [target_col]]

## Data tabulations and summarization:

In [52]:
# put id variables first:
X = X[id_variables + list(set(X.columns) - set(id_variables))]
X

,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
1,2000,S06037,0.242170,0.001574,0.016090,0.000425,15007.942937,1.0,0.479,0.074,...,0.0,1.40,35657.0,734.600003,0.006147,0.0,0.0,0.188,0.001285,0.004027
2,2000,S34001,0.239891,0.000351,0.020345,0.000702,929.185718,1.0,0.663,0.055,...,0.0,8.00,994.0,0.000000,0.012680,0.0,0.0,0.346,0.000879,0.002085
3,2000,S34003,0.239891,0.000351,0.020345,0.000702,1600.476204,1.0,0.672,0.047,...,0.0,8.00,3848.0,19.000000,0.012680,0.0,0.0,0.262,0.000879,0.002085
4,2000,S34005,0.239891,0.000351,0.020345,0.000702,1170.000004,1.0,0.774,0.053,...,0.0,8.00,1710.0,6.000000,0.012680,0.0,0.0,0.311,0.000879,0.002085
5,2000,S34007,0.239891,0.000351,0.020345,0.000702,769.100006,1.0,0.700,0.056,...,0.0,8.00,1452.0,11.800000,0.012680,0.0,0.0,0.321,0.000879,0.002085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.238576,0.001352,0.021517,0.000446,805.000000,1.0,0.303,0.060,...,0.0,0.00,4938.0,103.000000,0.006155,0.0,0.0,0.838,0.001512,0.000722
2469,2022,S36061,0.238576,0.001352,0.021517,0.000446,1675.000000,1.0,0.243,0.066,...,0.0,0.00,13303.0,1460.000000,0.006155,0.0,0.0,0.885,0.001512,0.000722
2470,2022,S36081,0.238576,0.001352,0.021517,0.000446,4914.000000,1.0,0.452,0.057,...,0.0,0.00,3772.0,3.000000,0.006155,0.0,0.0,0.828,0.001512,0.000722
2471,2022,S36085,0.238576,0.001352,0.021517,0.000446,289.000000,1.0,0.686,0.061,...,0.0,0.00,1036.0,12.000000,0.006155,0.0,0.0,0.887,0.001512,0.000722


In [53]:
list(X.columns)

['Year',
 'FIPS_2020',
 'PCE: Personal consumption expenditures',
 'IP: Wood products',
 'PCE: Other nondurable goods',
 'IP: Warehousing and storage',
 'CBP: Heavy and Civil Engineering Construction (237)',
 'Yard Waste Included Unknown',
 'DEM: Owner-Occupied Housing',
 'DEM: 20-24 years',
 'CBP: Food and Beverage Retailers (445)',
 'IP: Federal general government (defense - GFGD)',
 'CBP: Web Search Portals, Libraries, Archives, and Other Information Services (519)',
 'POL: Littering Fine',
 'CBP: Fishing, Hunting and Trapping (114)',
 'CBP: Petroleum and Coal Products Manufacturing (324)',
 'Food Waste Included Unknown',
 'CBP: Leather and Allied Product Manufacturing (316)',
 'POL: Plastic Bag Ban',
 'PointOfMeasurement_Self-Haul',
 'DEM: 10-14 years',
 'CBP: Wood Product Manufacturing (321)',
 'IP: Ambulatory health care services',
 'CBP: Health Care and Social Assistance (62)',
 'CBP: Arts, Entertainment, and Recreation (71)',
 'CBP: Repair and Maintenance (811)',
 'PCE: Transpo

In [54]:
X[id_variables+['Recycling Included',
 'Disposal Included',
 'Yard Waste Included',
 'Food Waste Included',
 'ActivityConsumedBy_Compost',
 'ActivityConsumedBy_Incinerator',
 'ActivityConsumedBy_Landfill',
 'ActivityConsumedBy_MRF',
 'ActivityConsumedBy_Recycling',
 'ActivityConsumedBy_Unknown',
 'ActivityProducedBy_C&D',
 'ActivityProducedBy_Commercial',
 'ActivityProducedBy_Industrial',
 'ActivityProducedBy_Institutional',
 'ActivityProducedBy_Landscape',
 'ActivityProducedBy_Residential',
 'ActivityProducedBy_Unspecified MSW',
 'PointOfMeasurement_Compost',
 'PointOfMeasurement_Composting',
 'PointOfMeasurement_Curbside Collection',
 'PointOfMeasurement_Incinerator/Combustion',
 'PointOfMeasurement_Landfill',
 'PointOfMeasurement_MRF',
 'PointOfMeasurement_Material Recovery Facility',
 'PointOfMeasurement_Recycling',
 'PointOfMeasurement_Self-Haul',
 'PointOfMeasurement_Transfer Station',
 'PointOfMeasurement_Unspecified Waste Facility',
 'PointOfMeasurement_Waste Haulers',]]

,Year,FIPS_2020,Recycling Included,Disposal Included,Yard Waste Included,Food Waste Included,ActivityConsumedBy_Compost,ActivityConsumedBy_Incinerator,ActivityConsumedBy_Landfill,ActivityConsumedBy_MRF,...,PointOfMeasurement_Curbside Collection,PointOfMeasurement_Incinerator/Combustion,PointOfMeasurement_Landfill,PointOfMeasurement_MRF,PointOfMeasurement_Material Recovery Facility,PointOfMeasurement_Recycling,PointOfMeasurement_Self-Haul,PointOfMeasurement_Transfer Station,PointOfMeasurement_Unspecified Waste Facility,PointOfMeasurement_Waste Haulers
1,2000,S06037,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2000,S34001,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2000,S34003,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2000,S34005,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,2000,S34007,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2469,2022,S36061,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2470,2022,S36081,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2471,2022,S36085,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
y[id_variables + [target_col]]

,Year,FIPS_2020,Waste Generation Amount
1,2000,S06037,11384354.0
2,2000,S34001,329432.0
3,2000,S34003,1040243.0
4,2000,S34005,469772.0
5,2000,S34007,493305.0
...,...,...,...
2468,2022,S36047,831039.0
2469,2022,S36061,395511.3
2470,2022,S36081,711611.3
2471,2022,S36085,182057.8


In [56]:
CBP_vars=['CBP: Real Estate and Rental and Leasing (53)',
 'CBP: Fabricated Metal Product Manufacturing (332)',
 'CBP: Water Transportation (483)',
 'CBP: Management of Companies and Enterprises (55)',
 'CBP: Food Manufacturing (311)',
 'CBP: Nonmetallic Mineral Product Manufacturing (327)',
 'CBP: Apparel Manufacturing (315)',
 'CBP: Personal and Laundry Services (812)',
 'CBP: Heavy and Civil Engineering Construction (237)',
 'CBP: Scenic and Sightseeing Transportation (487)',
 'CBP: Merchant Wholesalers, Durable Goods (423)',
 'CBP: Specialty Trade Contractors (238)',
 'CBP: Ambulatory Health Care Services (621)',
 'CBP: Computer and Electronic Product Manufacturing (334)',
 'CBP: Wholesale Trade (42)',
 'CBP: Mining (except Oil and Gas) (212)',
 'CBP: Waste Management and Remediation Services (562)',
 'CBP: Amusement, Gambling, and Recreation Industries (713)',
 'CBP: Lessors of Nonfinancial Intangible Assets (except Copyrighted Works) (533)',
 'CBP: Accommodation and Food Services (72)',
 'CBP: Wholesale Trade Agents and Brokers (425)',
 'CBP: Petroleum and Coal Products Manufacturing (324)',
 'CBP: Transportation Equipment Manufacturing (336)',
 'CBP: Support Activities for Transportation (488)',
 'CBP: Miscellaneous Manufacturing (339)',
 'CBP: Couriers and Messengers (492)',
 'CBP: Computing Infrastructure Providers, Data Processing, Web Hosting (518)',
 'CBP: Mining, Quarrying, and Oil and Gas Extraction (21)',
 'CBP: Printing and Related Support Activities (323)',
 'CBP: Fishing, Hunting and Trapping (114)',
 'CBP: Transit and Ground Passenger Transportation (485)',
 'CBP: Agriculture, Forestry, Fishing and Hunting - (11 excl. 111, 112)',
 'CBP: Web Search Portals, Libraries, Archives, and Other Information Services (519)',
 'CBP: Air Transportation (481)',
 'CBP: Administrative and Support and Waste Management and Remediation Services (56)',
 'CBP: Furniture and Related Product Manufacturing (337)',
 'CBP: Utilities (22)',
 'CBP: Educational Services (611)',
 'CBP: Telecommunications (517)',
 'CBP: Truck Transportation (484)',
 'CBP: Construction (23)',
 'CBP: Information (51)',
 'CBP: Rental and Leasing Services (532)',
 'CBP: Monetary Authorities-Central Bank (521)',
 'CBP: Support Activities for Mining (213)',
 'CBP: Utilities (221)',
 'CBP: Professional, Scientific, and Technical Services (541)',
 'CBP: Museums, Historical Sites, and Similar Institutions (712)',
 'CBP: Oil and Gas Extraction (211)',
 'CBP: Administrative and Support Services (561)',
 'CBP: Manufacturing (31-33 )',
 'CBP: Arts, Entertainment, and Recreation (71)',
 'CBP: Building Material and Garden Equipment and Supplies Dealers (444)',
 'CBP: Insurance Carriers and Related Activities (524)',
 'CBP: Nursing and Residential Care Facilities (623)',
 'CBP: Textile Mills (313)',
 'CBP: Merchant Wholesalers, Nondurable Goods (424)',
 'CBP: Finance and Insurance (52 excluding 525110, 525120, 525190, 525920)',
 'CBP: Electrical Equipment, Appliance, and Component Manufacturing (335)',
 'CBP: Credit Intermediation and Related Activities (522)',
 'CBP: Accommodation (721)',
 'CBP: Chemical Manufacturing (325)',
 'CBP: Real Estate (531)',
 'CBP: Pipeline Transportation (486)',
 'CBP: Securities, Commodity Contracts, and Other Financial Investments and Related Activities (523)',
 'CBP: Support Activities for Crop Production (115)',
 'CBP: Repair and Maintenance (811)',
 'CBP: Forestry and Logging (113)',
 'CBP: Retail Trade (44-45 )',
 'CBP: Management of Companies and Enterprises (551)',
 'CBP: Religious, Grantmaking, Civic, Professional, and Similar Organizations (813)',
 'CBP: Food and Beverage Retailers (445)',
 'CBP: Funds, Trusts, and Other Financial Vehicles (525)',
 'CBP: Food Services and Drinking Places (722)',
 'CBP: Other Services Except Public Administration (81, excl. 814)',
 'CBP: Warehousing and Storage (493)',
 'CBP: Textile Product Mills (314)',
 'CBP: Educational Services (61)',
 'CBP: Beverage and Tobacco Product Manufacturing (312)',
 'CBP: Primary Metal Manufacturing (331)',
 'CBP: Performing Arts, Spectator Sports, and Related Industries (711)',
 'CBP: Transportation and Warehousing (48-49, excl. 482, 491)',
 'CBP: Plastics and Rubber Products Manufacturing (326)',
 'CBP: Hospitals (622)',
 'CBP: Motor Vehicle and Parts Dealers (441)',
 'CBP: Motion Picture and Sound Recording Industries (512)',
 'CBP: Wood Product Manufacturing (321)',
 'CBP: Health Care and Social Assistance (62)',
 'CBP: Leather and Allied Product Manufacturing (316)',
 'CBP: Machinery Manufacturing (333)',
 'CBP: Paper Manufacturing (322)',
 'CBP: Construction of Buildings (236)',
 'CBP: Professional, Scientific, and Technical Services (541 excl 541120)',
 'CBP: Social Assistance (624)',]
X[ id_variables + CBP_vars]

,Year,FIPS_2020,CBP: Real Estate and Rental and Leasing (53),CBP: Fabricated Metal Product Manufacturing (332),CBP: Water Transportation (483),CBP: Management of Companies and Enterprises (55),CBP: Food Manufacturing (311),CBP: Nonmetallic Mineral Product Manufacturing (327),CBP: Apparel Manufacturing (315),CBP: Personal and Laundry Services (812),...,CBP: Motor Vehicle and Parts Dealers (441),CBP: Motion Picture and Sound Recording Industries (512),CBP: Wood Product Manufacturing (321),CBP: Health Care and Social Assistance (62),CBP: Leather and Allied Product Manufacturing (316),CBP: Machinery Manufacturing (333),CBP: Paper Manufacturing (322),CBP: Construction of Buildings (236),"CBP: Professional, Scientific, and Technical Services (541 excl 541120)",CBP: Social Assistance (624)
1,2000,S06037,86025.400013,69706.0,3434.0,83151.0,43050.0,9939.0,71487.0,50795.0,...,51018.652037,73713.0,7703.0,372844.0,3878.0,28867.500019,12582.0,35546.876739,402582.0,46624.0
2,2000,S34001,2083.000000,92.0,7.0,915.0,489.0,922.0,63.5,1401.0,...,1876.131868,110.0,185.0,12499.0,0.0,165.000000,0.0,1112.485732,4777.0,1729.0
3,2000,S34003,9657.000001,4911.0,710.0,32630.0,4245.0,532.0,1268.0,6877.0,...,6159.141035,1149.0,233.0,48694.0,882.0,3563.000000,4483.0,4579.809589,29243.0,4631.0
4,2000,S34005,2752.000000,1828.0,4.0,4152.0,770.0,619.0,327.0,2244.0,...,3498.879121,510.0,275.0,19539.0,4.0,1888.500000,398.0,2263.000038,12212.5,2957.0
5,2000,S34007,2998.200001,2016.0,331.0,6362.0,2437.0,826.0,309.0,2763.0,...,2710.214288,543.0,362.0,30728.0,9.0,1413.500000,1325.0,2359.466711,13495.5,3955.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,18566.000000,1623.0,286.0,3331.0,5234.0,723.0,1106.0,11607.0,...,2310.000000,2136.0,271.0,282895.0,64.0,288.000000,58.0,8186.000000,26121.0,79038.0
2469,2022,S36061,70696.000000,367.0,416.0,83488.0,2133.0,40.0,2473.0,25915.0,...,1110.000000,22568.0,2.0,264995.0,76.0,301.000000,21.0,16201.000000,349141.0,64146.0
2470,2022,S36081,13522.000000,2169.0,67.0,6645.0,2976.0,1112.0,1055.0,10069.0,...,3528.000000,728.0,266.0,176007.0,27.0,337.000000,71.0,12027.000000,16758.0,43666.0
2471,2022,S36085,966.000000,39.0,733.0,280.0,177.0,56.0,0.0,2507.0,...,851.000000,192.0,0.0,30828.0,0.0,0.000000,262.0,1874.000000,3316.0,5956.0


In [57]:
PCE_vars=[ 'PCE: Clothing and footwear',
 'PCE: Financial services and insurance',
 'PCE: Food and beverages purchased for off-premises consumption',
 'PCE: Food services and accommodations',
 'PCE: Furnishings and durable household equipment',
 'PCE: Gasoline and other energy goods',
 'PCE: Gross output of nonprofit institutions',
 'PCE: Health care',
 'PCE: Housing and utilities',
 'PCE: Sales of goods and services by nonprofit institutions',
 'PCE: Motor vehicles and parts',
 'PCE: Other durable goods',
 'PCE: Other nondurable goods',
 'PCE: Other services',
 'PCE: Recreation services',
 'PCE: Recreational goods and vehicles',
 'PCE: Transportation services',
 'PCE: Durable goods',
 'PCE: Final consumption expenditures of nonprofit institutions serving households (NPISHs)',
 'PCE: Household consumption expenditures (for services)',
 'PCE: Nondurable goods',
 'PCE: Goods',
 'PCE: Services',
 'PCE: Personal consumption expenditures',]
X[id_variables+PCE_vars]

,Year,FIPS_2020,PCE: Clothing and footwear,PCE: Financial services and insurance,PCE: Food and beverages purchased for off-premises consumption,PCE: Food services and accommodations,PCE: Furnishings and durable household equipment,PCE: Gasoline and other energy goods,PCE: Gross output of nonprofit institutions,PCE: Health care,...,PCE: Recreation services,PCE: Recreational goods and vehicles,PCE: Transportation services,PCE: Durable goods,PCE: Final consumption expenditures of nonprofit institutions serving households (NPISHs),PCE: Household consumption expenditures (for services),PCE: Nondurable goods,PCE: Goods,PCE: Services,PCE: Personal consumption expenditures
1,2000,S06037,0.010557,0.019777,0.018278,0.014811,0.007262,0.005074,0.020124,0.029621,...,0.011413,0.010178,0.012103,0.032492,0.004463,0.155217,0.049998,0.082490,0.159680,0.242170
2,2000,S34001,0.011826,0.022108,0.018497,0.011753,0.007477,0.004517,0.028958,0.030211,...,0.008591,0.007619,0.009522,0.030049,0.008739,0.145918,0.055184,0.085233,0.154658,0.239891
3,2000,S34003,0.011826,0.022108,0.018497,0.011753,0.007477,0.004517,0.028958,0.030211,...,0.008591,0.007619,0.009522,0.030049,0.008739,0.145918,0.055184,0.085233,0.154658,0.239891
4,2000,S34005,0.011826,0.022108,0.018497,0.011753,0.007477,0.004517,0.028958,0.030211,...,0.008591,0.007619,0.009522,0.030049,0.008739,0.145918,0.055184,0.085233,0.154658,0.239891
5,2000,S34007,0.011826,0.022108,0.018497,0.011753,0.007477,0.004517,0.028958,0.030211,...,0.008591,0.007619,0.009522,0.030049,0.008739,0.145918,0.055184,0.085233,0.154658,0.239891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.006926,0.018230,0.016779,0.016847,0.005689,0.003837,0.032264,0.041237,...,0.007851,0.006603,0.008155,0.021971,0.009417,0.158128,0.049060,0.071031,0.167546,0.238576
2469,2022,S36061,0.006926,0.018230,0.016779,0.016847,0.005689,0.003837,0.032264,0.041237,...,0.007851,0.006603,0.008155,0.021971,0.009417,0.158128,0.049060,0.071031,0.167546,0.238576
2470,2022,S36081,0.006926,0.018230,0.016779,0.016847,0.005689,0.003837,0.032264,0.041237,...,0.007851,0.006603,0.008155,0.021971,0.009417,0.158128,0.049060,0.071031,0.167546,0.238576
2471,2022,S36085,0.006926,0.018230,0.016779,0.016847,0.005689,0.003837,0.032264,0.041237,...,0.007851,0.006603,0.008155,0.021971,0.009417,0.158128,0.049060,0.071031,0.167546,0.238576


In [58]:
IP_vars=[
 'IP: Forestry, fishing, and related activities',
 'IP: Oil and gas extraction',
 'IP: Mining, except oil and gas',
 'IP: Support activities for mining',
 'IP: Utilities',
 'IP: Construction',
 'IP: Food and beverage and tobacco products',
 'IP: Textile mills and textile product mills',
 'IP: Apparel and leather and allied products',
 'IP: Wood products',
 'IP: Paper products',
 'IP: Printing and related support activities',
 'IP: Petroleum and coal products',
 'IP: Chemical products',
 'IP: Plastics and rubber products',
 'IP: Nonmetallic mineral products',
 'IP: Primary metals',
 'IP: Fabricated metal products',
 'IP: Machinery',
 'IP: Computer and electronic products',
 'IP: Electrical equipment, appliances, and components',
 'IP: Motor vehicles, bodies and trailers, and parts',
 'IP: Other transportation equipment',
 'IP: Furniture and related products',
 'IP: Miscellaneous manufacturing',
 'IP: Wholesale trade',
 'IP: Motor vehicle and parts dealers',
 'IP: Food and beverage stores',
 'IP: Air transportation',
 'IP: Water transportation',
 'IP: Truck transportation',
 'IP: Transit and ground passenger transportation',
 'IP: Pipeline transportation',
 'IP: Other transportation and support activities',
 'IP: Warehousing and storage',
 'IP: Motion picture and sound recording industries',
 'IP: Federal Reserve banks, credit intermediation, and related activities',
 'IP: Securities, commodity contracts, and investments',
 'IP: Insurance carriers and related activities',
 'IP: Funds, trusts, and other financial vehicles',
 'IP: Rental and leasing services and lessors of intangible assets',
 'IP: Legal services',
 'IP: Miscellaneous professional, scientific, and technical services',
 'IP: Computer systems design and related services',
 'IP: Management of companies and enterprises',
 'IP: Administrative and support services',
 'IP: Waste management and remediation services',
 'IP: Educational services',
 'IP: Ambulatory health care services',
 'IP: Hospitals',
 'IP: Nursing and residential care facilities',
 'IP: Social assistance',
 'IP: Performing arts, spectator sports, museums, and related activities',
 'IP: Amusements, gambling, and recreation industries',
 'IP: Accommodation',
 'IP: Food services and drinking places',
 'IP: Other services, except government',
 'IP: Federal government enterprises (GFE)',
 'IP: Federal general government (defense - GFGD)',
 'IP: Federal general government (nondefense - GFGN)',
 'IP: State and local government enterprises (GSLE)',
 'IP: State and local general government (GSLG)',
 'IP: Housing (531 real estate)',
 'IP: Other real estate (531 - not housing)']
X[id_variables+IP_vars]

,Year,FIPS_2020,"IP: Forestry, fishing, and related activities",IP: Oil and gas extraction,"IP: Mining, except oil and gas",IP: Support activities for mining,IP: Utilities,IP: Construction,IP: Food and beverage and tobacco products,IP: Textile mills and textile product mills,...,IP: Accommodation,IP: Food services and drinking places,"IP: Other services, except government",IP: Federal government enterprises (GFE),IP: Federal general government (defense - GFGD),IP: Federal general government (nondefense - GFGN),IP: State and local government enterprises (GSLE),IP: State and local general government (GSLG),IP: Housing (531 real estate),IP: Other real estate (531 - not housing)
1,2000,S06037,0.007287,0.016507,0.000223,0.001405,0.016443,0.000000,0.109470,0.003314,...,0.001785,0.009808,0.010016,0.002154,0.009897,0.004027,0.010853,0.041523,0.009794,0.003516
2,2000,S34001,0.000350,0.000000,0.000829,0.000000,0.021702,0.028979,0.047867,0.003470,...,0.004293,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969
3,2000,S34003,0.000350,0.000000,0.000829,0.000000,0.021702,0.028979,0.047867,0.003470,...,0.004293,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969
4,2000,S34005,0.000350,0.000000,0.000829,0.000000,0.021702,0.028979,0.047867,0.003470,...,0.004293,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969
5,2000,S34007,0.000350,0.000000,0.000829,0.000000,0.021702,0.028979,0.047867,0.003470,...,0.004293,0.007354,0.007998,0.001122,0.000896,0.002085,0.011442,0.046237,0.013490,0.005969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.000185,0.000014,0.000583,0.000022,0.009815,0.050063,0.035104,0.001437,...,0.002943,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240
2469,2022,S36061,0.000185,0.000014,0.000583,0.000022,0.009815,0.050063,0.035104,0.001437,...,0.002943,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240
2470,2022,S36081,0.000185,0.000014,0.000583,0.000022,0.009815,0.050063,0.035104,0.001437,...,0.002943,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240
2471,2022,S36085,0.000185,0.000014,0.000583,0.000022,0.009815,0.050063,0.035104,0.001437,...,0.002943,0.012026,0.011605,0.000523,0.003765,0.000722,0.013206,0.059165,0.021883,0.012240


In [59]:
AGG_vars=['AGG: Total Irrigated Vegetable Area (acres)',
 'AGG: Total Nursery in Open Production Area (acres)',
 'AGG: Total Nursery in Protection Production Area (sq. ft.)']
X[id_variables+AGG_vars]

,Year,FIPS_2020,AGG: Total Irrigated Vegetable Area (acres),AGG: Total Nursery in Open Production Area (acres),AGG: Total Nursery in Protection Production Area (sq. ft.)
1,2000,S06037,6174.6,982.0,5872794.4
2,2000,S34001,4931.4,0.0,205670.8
3,2000,S34003,102.8,0.0,0.0
4,2000,S34005,3601.6,1505.0,18877.2
5,2000,S34007,1562.6,101.8,0.0
...,...,...,...,...,...
2468,2022,S36047,14.0,4.0,0.0
2469,2022,S36061,0.0,0.0,0.0
2470,2022,S36081,2.0,0.0,0.0
2471,2022,S36085,0.0,0.0,0.0


In [60]:
DEM_vars=['DEM: $10,000 - $14,999',
 'DEM: $100,000 - $149,999',
 'DEM: $15,000 - $24,999',
 'DEM: $150,000 - $199,999',
 'DEM: $200,000 and over',
 'DEM: $25,000 - $34,999',
 'DEM: $35,000 - $49,999',
 'DEM: $50,000 - $74,999',
 'DEM: $75,000 - $99,999',
 'DEM: 10-14 years',
 'DEM: 15-19 years',
 'DEM: 20-24 years',
 'DEM: 25-34 years',
 'DEM: 35-44 years',
 'DEM: 45-54 years',
 'DEM: 5-9 years',
 'DEM: 55-59 years',
 'DEM: 60-64 years',
 'DEM: 65-74 years',
 'DEM: 75-84 years',
 'DEM: 85+ years',
 'DEM: Bachelors Degree',
 'DEM: High School Degree',
 'DEM: Owner-Occupied Housing',
 'DEM: Proportion Below Poverty Line',
 'DEM: Under 5 years',
 'DEM: Unemployment Rate',
 'DEM: less than $10,000',]
X[id_variables+DEM_vars]



,Year,FIPS_2020,"DEM: $10,000 - $14,999","DEM: $100,000 - $149,999","DEM: $15,000 - $24,999","DEM: $150,000 - $199,999","DEM: $200,000 and over","DEM: $25,000 - $34,999","DEM: $35,000 - $49,999","DEM: $50,000 - $74,999",...,DEM: 65-74 years,DEM: 75-84 years,DEM: 85+ years,DEM: Bachelors Degree,DEM: High School Degree,DEM: Owner-Occupied Housing,DEM: Proportion Below Poverty Line,DEM: Under 5 years,DEM: Unemployment Rate,"DEM: less than $10,000"
1,2000,S06037,0.065,0.088,0.127,0.028,0.035,0.122,0.151,0.178,...,0.052,0.034,0.011,0.161,0.188,0.479,0.179,0.077,0.050,0.105
2,2000,S34001,0.056,0.074,0.123,0.014,0.018,0.130,0.170,0.215,...,0.073,0.047,0.015,0.128,0.346,0.663,0.105,0.064,0.048,0.085
3,2000,S34003,0.036,0.155,0.076,0.059,0.073,0.084,0.132,0.190,...,0.078,0.056,0.019,0.240,0.262,0.672,0.050,0.062,0.026,0.050
4,2000,S34005,0.034,0.137,0.082,0.035,0.029,0.102,0.154,0.232,...,0.069,0.044,0.013,0.192,0.311,0.774,0.047,0.063,0.026,0.039
5,2000,S34007,0.053,0.098,0.110,0.026,0.023,0.113,0.158,0.212,...,0.065,0.046,0.014,0.156,0.321,0.700,0.104,0.067,0.039,0.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.058,0.153,0.078,0.088,0.148,0.070,0.093,0.134,...,0.084,0.042,0.019,0.404,0.838,0.303,0.190,0.068,0.073,0.068
2469,2022,S36061,0.052,0.138,0.065,0.091,0.270,0.051,0.068,0.105,...,0.094,0.054,0.025,0.628,0.885,0.243,0.158,0.045,0.068,0.072
2470,2022,S36081,0.035,0.184,0.061,0.107,0.124,0.069,0.092,0.153,...,0.095,0.048,0.021,0.348,0.828,0.452,0.117,0.058,0.070,0.049
2471,2022,S36085,0.034,0.186,0.050,0.129,0.170,0.052,0.078,0.134,...,0.099,0.048,0.019,0.353,0.887,0.686,0.104,0.055,0.054,0.050


In [61]:
OtherRate_vars = [ 'INST: Prop. of population that attends church', 'POL: Required recycling goal']
X[id_variables+OtherRate_vars]

,Year,FIPS_2020,INST: Prop. of population that attends church,POL: Required recycling goal
1,2000,S06037,0.514205,0.5
2,2000,S34001,0.441706,0.6
3,2000,S34003,0.723707,0.6
4,2000,S34005,0.450310,0.6
5,2000,S34007,0.545684,0.6
...,...,...,...,...
2468,2022,S36047,0.550999,0.0
2469,2022,S36061,0.574054,0.0
2470,2022,S36081,0.504628,0.0
2471,2022,S36085,0.697451,0.0


In [62]:
DEM_cont_vars=[ 'DEM: Population',
'DEM: Population Density (people per sq mile)',]
X[id_variables+DEM_cont_vars]

,Year,FIPS_2020,DEM: Population,DEM: Population Density (people per sq mile)
1,2000,S06037,9519338.0,2345.079253
2,2000,S34001,252552.0,454.627606
3,2000,S34003,884118.0,3797.953503
4,2000,S34005,423394.0,529.711294
5,2000,S34007,508932.0,2299.135337
...,...,...,...,...
2468,2022,S36047,2679620.0,38624.039667
2469,2022,S36061,1645867.0,72645.965749
2470,2022,S36081,2360826.0,21713.936205
2471,2022,S36085,492925.0,8569.181023


In [63]:
ACT_vars = ['ActivityConsumedBy_Compost',
 'ActivityConsumedBy_Incinerator',
 'ActivityConsumedBy_Landfill',
 'ActivityConsumedBy_MRF',
 'ActivityConsumedBy_Recycling',
 'ActivityConsumedBy_Unknown',
 'ActivityProducedBy_C&D',
 'ActivityProducedBy_Commercial',
 'ActivityProducedBy_Industrial',
 'ActivityProducedBy_Institutional',
 'ActivityProducedBy_Landscape',
 'ActivityProducedBy_Residential',
 'ActivityProducedBy_Unspecified MSW',]
X[ACT_vars].value_counts()

ActivityConsumedBy_Compost  ActivityConsumedBy_Incinerator  ActivityConsumedBy_Landfill  ActivityConsumedBy_MRF  ActivityConsumedBy_Recycling  ActivityConsumedBy_Unknown  ActivityProducedBy_C&D  ActivityProducedBy_Commercial  ActivityProducedBy_Industrial  ActivityProducedBy_Institutional  ActivityProducedBy_Landscape  ActivityProducedBy_Residential  ActivityProducedBy_Unspecified MSW
0.0                         0.0                             1.0                          0.0                     0.0                           0.0                         0.0                     0.0                            0.0                            0.0                               0.0                           0.0                             1.0                                   698
                                                                                                                 1.0                           0.0                         0.0                     0.0                            0.0                            0.0                               0.0                           0.0                             1.0                                   593
                                                                                                                 0.0                           0.0                         0.0                     1.0                            1.0                            1.0                               0.0                           1.0                             0.0                                   498
                                                                                                                 1.0                           0.0                         1.0                     0.0                            1.0                            0.0                               0.0                           0.0                             1.0                                   260
                                                                                                                 0.0                           0.0                         0.0                     1.0                            0.0                            0.0                               0.0                           1.0                             0.0                                   101
                                                                                         1.0                     0.0                           0.0                         0.0                     0.0                            0.0                            0.0                               0.0                           0.0                             1.0                                    46
                                                                                         0.0                     1.0                           0.0                         0.0                     1.0                            0.0                            1.0                               0.0                           1.0                             0.0                                    22
                                                                                                                                                                                                                                  1.0                            0.0                               0.0                           1.0                             0.0                                    21
                                                                                                                 0.0                           0.0                         0.0                     1.0                            0.0                            1.0                               0.0                           1.0                             0.0                                    18
                                                         

In [64]:
INF_Vars=[
    'INF: # MRFs',
 'INF: # MSW Landfills',
 'INF: # TSs',
 'INST: Number of churches',
 'INST: University enrollment']
X[id_variables + INF_Vars]

,Year,FIPS_2020,INF: # MRFs,INF: # MSW Landfills,INF: # TSs,INST: Number of churches,INST: University enrollment
1,2000,S06037,12.0,9.0,129.0,4306.0,871464.0
2,2000,S34001,0.0,1.0,1.0,265.0,14600.0
3,2000,S34003,0.0,1.0,8.0,508.0,39323.0
4,2000,S34005,1.0,1.0,2.0,391.0,10272.0
5,2000,S34007,1.0,1.0,0.0,292.0,23908.0
...,...,...,...,...,...,...,...
2468,2022,S36047,1.0,0.0,8.0,1192.0,98403.0
2469,2022,S36061,0.0,0.0,2.0,734.0,343591.0
2470,2022,S36081,0.0,0.0,8.0,841.0,100487.0
2471,2022,S36085,0.0,0.0,2.0,590.0,18054.0


In [65]:
POL_vars=['POL: Bottle Bill Price',
 'POL: Landfill tax',
 'POL: Tipping Fee',
    ]
X[id_variables + POL_vars]

,Year,FIPS_2020,POL: Bottle Bill Price,POL: Landfill tax,POL: Tipping Fee
1,2000,S06037,0.05,1.40,59.681429
2,2000,S34001,0.00,8.00,77.154286
3,2000,S34003,0.00,8.00,77.154286
4,2000,S34005,0.00,8.00,77.154286
5,2000,S34007,0.00,8.00,77.154286
...,...,...,...,...,...
2468,2022,S36047,0.05,0.00,66.170000
2469,2022,S36061,0.05,0.00,66.170000
2470,2022,S36081,0.05,0.00,66.170000
2471,2022,S36085,0.05,0.00,66.170000


In [66]:

POL_binary_vars = [
 'POL: Require user fee',
 'POL: Subsidize user fee',
 'POL: Littering Fine',
 'POL: Littering Penalties',
 'POL: Littering is Criminal',
 'POL: Plastic Bag Ban',
 'POL: Promote user fee',
 'POL: Require curbside recycling',
]
X[POL_binary_vars].value_counts()

POL: Require user fee  POL: Subsidize user fee  POL: Littering Fine  POL: Littering Penalties  POL: Littering is Criminal  POL: Plastic Bag Ban  POL: Promote user fee  POL: Require curbside recycling
0.0                    0.0                      1.0                  1.0                       1.0                         0.0                   1.0                    1.0                                864
                       1.0                      1.0                  1.0                       1.0                         0.0                   0.0                    0.0                                754
                       0.0                      1.0                  1.0                       0.0                         0.0                   0.0                    0.0                                502
                       1.0                      1.0                  1.0                       0.0                         0.0                   0.0                    0.0                                 91
                       0.0                      1.0                  1.0                       1.0                         0.0                   1.0                    0.0                                 80
                       1.0                      1.0                  1.0                       1.0                         0.0                   0.0                    1.0                                 68
                       0.0                      1.0                  1.0                       0.0                         0.0                   1.0                    1.0                                 30
                                                                                               1.0                         0.0                   0.0                    1.0                                 30
                                                                                                                                                                        0.0                                 24
                                                                                                                           1.0                   1.0                    1.0                                 10
1.0                    0.0                      1.0                  1.0                       1.0                         0.0                   0.0                    1.0                                  8
0.0                    0.0                      1.0                  1.0                       1.0                         1.0                   0.0                    0.0                                  7
                       1.0                      1.0                  1.0                       1.0                         0.0                   1.0                    0.0                                  2
Name: count, dtype: int64

In [67]:

POM_vars=[
 'PointOfMeasurement_Compost',
 'PointOfMeasurement_Composting',
 'PointOfMeasurement_Curbside Collection',
 'PointOfMeasurement_Incinerator/Combustion',
 'PointOfMeasurement_Landfill',
 'PointOfMeasurement_MRF',
 'PointOfMeasurement_Material Recovery Facility',
 'PointOfMeasurement_Recycling',
 'PointOfMeasurement_Self-Haul',
 'PointOfMeasurement_Transfer Station',
 'PointOfMeasurement_Unspecified Waste Facility',
 'PointOfMeasurement_Waste Haulers']
X[POM_vars].value_counts()

PointOfMeasurement_Compost  PointOfMeasurement_Composting  PointOfMeasurement_Curbside Collection  PointOfMeasurement_Incinerator/Combustion  PointOfMeasurement_Landfill  PointOfMeasurement_MRF  PointOfMeasurement_Material Recovery Facility  PointOfMeasurement_Recycling  PointOfMeasurement_Self-Haul  PointOfMeasurement_Transfer Station  PointOfMeasurement_Unspecified Waste Facility  PointOfMeasurement_Waste Haulers
0.0                         0.0                            0.0                                     0.0                                        0.0                          0.0                     0.0                                            0.0                           0.0                           0.0                                  1.0                                            0.0                                 1627
                                                                                                                                              1.0                          0.0                     0.0                                            0.0                           0.0                           0.0                                  0.0                                            0.0                                  627
                                                           1.0                                     0.0                                        0.0                          0.0                     0.0                                            0.0                           0.0                           0.0                                  0.0                                            0.0                                  120
                                                           0.0                                     0.0                                        1.0                          1.0                     0.0                                            0.0                           0.0                           0.0                                  0.0                                            0.0                                   46
                                                                                                                                                                           0.0                     0.0                                            1.0                           0.0                           0.0                                  0.0                                            0.0                                   14
                                                                                                                                                                                                                                                  0.0                           0.0                           0.0                                  1.0                                            0.0                                   12
1.0                         0.0                            0.0                                     0.0                                        1.0                          0.0                     0.0                                            1.0                           0.0                           0.0                                  0.0                                            0.0                                    8
0.0                         0.0                            0.0                                     0.0                                        0.0                          0.0                     1.0                                            0.0                           0.0                           1.0                                  0.0                                            0.0                                    5
                            1.0                            0.0                                     0.0                                        1.0                    

In [68]:
X[['Disposal Included',
 'Food Waste Included',
 'Recycling Included',
 'Yard Waste Included']].value_counts()

Disposal Included  Food Waste Included  Recycling Included  Yard Waste Included
1.0                0.0                  1.0                 0.0                    1508
                                        0.0                 0.0                     693
                   1.0                  0.0                 0.0                     120
                                        1.0                 1.0                     111
                   0.0                  0.0                 1.0                      15
                   1.0                  0.0                 1.0                      12
                   0.0                  1.0                 1.0                      10
                   1.0                  1.0                 0.0                       1
Name: count, dtype: int64

In [69]:
INCLUSION_vars = ['Disposal Included',
 'Food Waste Included',
 'Food Waste Included Unknown',
 'Recycling Included',
 'Yard Waste Included',
 'Yard Waste Included Unknown']
X[INCLUSION_vars].value_counts()

Disposal Included  Food Waste Included  Food Waste Included Unknown  Recycling Included  Yard Waste Included  Yard Waste Included Unknown
1.0                0.0                  1.0                          1.0                 0.0                  1.0                            1496
                                                                     0.0                 0.0                  1.0                             693
                   1.0                  0.0                          0.0                 0.0                  1.0                             120
                                                                     1.0                 1.0                  0.0                             111
                   0.0                  1.0                          0.0                 1.0                  0.0                              15
                                                                     1.0                 0.0                  0.0                              12
                   1.0                  0.0                          0.0                 1.0                  0.0                              12
                   0.0                  1.0                          1.0                 1.0                  0.0                               9
                                        0.0                          1.0                 1.0                  0.0                               1
                   1.0                  0.0                          1.0                 0.0                  0.0                               1
Name: count, dtype: int64

In [70]:
try:
    X = X.drop(columns=['is_duplicated'], axis=1)
except:
    pass

In [71]:
# Now we check that are columns are assigned to a group (outpt should be only the identifiers, since everything else was assigned to a group):
set(X.columns)-set(CBP_vars)-set(PCE_vars)-set(IP_vars)-set(AGG_vars)-set(DEM_vars)-set(DEM_cont_vars)-set(ACT_vars)-set(INF_Vars)-set(POL_vars)-set(POM_vars)-set(POL_binary_vars)-set(OtherRate_vars)-set(INCLUSION_vars)

{'FIPS_2020', 'Year'}

## Preprocessing/scaling:

In [72]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer, RobustScaler
import numpy as np

def create_preprocessor(X, continuous_vars, proportion_vars, binary_vars):
    """
    Create a preprocessor optimized for SVR:
    - Continuous variables: RobustScaler (handles outliers better)
    - Proportions (0-1): No transformation (already bounded)
    - Binary (0-1): No transformation
    """
    transformers = []

    # Continuous variables
    if continuous_vars:
        continuous_transformer = Pipeline([
            # ('scaler', RobustScaler())
        ('scaler', StandardScaler())
        ])
        transformers.append(('continuous', continuous_transformer, continuous_vars))

    # Proportion variables - already 0-1 bounded, no transformation needed
    if proportion_vars:
        transformers.append(('proportion', 'passthrough', proportion_vars))

    # Binary variables - no transformation needed
    if binary_vars:
        transformers.append(('binary', 'passthrough', binary_vars))

    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder='drop' # variables nor included in the list will be dropped, such as index variables
    )

    return preprocessor

# Define variable types
continuous_vars = list(set(CBP_vars) | set(AGG_vars) | set(DEM_cont_vars) | set(INF_Vars) | set(POL_vars))
proportion_vars = list(set(PCE_vars) | set(IP_vars) | set(DEM_vars) | set(OtherRate_vars))
binary_vars = list(set(ACT_vars) | set(POM_vars) | set(POL_binary_vars) | set(INCLUSION_vars))

# Print summary
print("Continuous variables:", len(continuous_vars))
print("Proportion variables:", len(proportion_vars))
print("Binary variables:", len(binary_vars))

# Create and fit preprocessor
preprocessor = create_preprocessor(X,
                                 continuous_vars=continuous_vars,
                                 proportion_vars=proportion_vars,
                                 binary_vars=binary_vars)

# Store column names
training_columns = continuous_vars + proportion_vars + binary_vars

preprocessor.fit(X)


Continuous variables: 107
Proportion variables: 118
Binary variables: 39


ColumnTransformer(transformers=[('continuous',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['CBP: Building Material and Garden Equipment '
                                  'and Supplies Dealers (444)',
                                  'CBP: Administrative and Support Services '
                                  '(561)',
                                  'POL: Tipping Fee',
                                  'CBP: Other Services Except Public '
                                  'Administration (81, excl. 814)',
                                  'CBP: Heavy and Civil Engineering '
                                  'Construction (237)',
                                  'CBP: Hospitals (62...
                                  'PointOfMeasurement_Material Recovery '
                                  'Facility',
                                  'POL: Subsidize user fee',
                                  'POL: Promote user fee',
                                  'PointOfMeasurement_Composting',
                                  'PointOfMeasurement_Unspecified Waste '
                                  'Facility',
                                  'ActivityProducedBy_Landscape',
                                  'PointOfMeasurement_Incinerator/Combustion',
                                  'ActivityProducedBy_Industrial',
                                  'PointOfMeasurement_Curbside Collection',
                                  'PointOfMeasurement_MRF',
                                  'ActivityProducedBy_C&D', ...])])

In [73]:
import joblib

# Save preprocessor for later use:
preprocessor_filename = folder_path_OUTPUT_VARIABLES + f"preprocessor_{name_suffix}.joblib"
print("Model saved to : ", preprocessor_filename)
joblib.dump(preprocessor, preprocessor_filename)

Model saved to :  /content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/preprocessor_KeepImp0_GenInter0_SmallSample0.joblib


['/content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/preprocessor_KeepImp0_GenInter0_SmallSample0.joblib']

In [74]:
# test loading preprocessor:
preprocessor_filename = folder_path_OUTPUT_VARIABLES + f"preprocessor_{name_suffix}.joblib"
loaded_preprocessor = joblib.load(preprocessor_filename)
loaded_preprocessor

ColumnTransformer(transformers=[('continuous',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['CBP: Building Material and Garden Equipment '
                                  'and Supplies Dealers (444)',
                                  'CBP: Administrative and Support Services '
                                  '(561)',
                                  'POL: Tipping Fee',
                                  'CBP: Other Services Except Public '
                                  'Administration (81, excl. 814)',
                                  'CBP: Heavy and Civil Engineering '
                                  'Construction (237)',
                                  'CBP: Hospitals (62...
                                  'PointOfMeasurement_Material Recovery '
                                  'Facility',
                                  'POL: Subsidize user fee',
                                  'POL: Promote user fee',
                                  'PointOfMeasurement_Composting',
                                  'PointOfMeasurement_Unspecified Waste '
                                  'Facility',
                                  'ActivityProducedBy_Landscape',
                                  'PointOfMeasurement_Incinerator/Combustion',
                                  'ActivityProducedBy_Industrial',
                                  'PointOfMeasurement_Curbside Collection',
                                  'PointOfMeasurement_MRF',
                                  'ActivityProducedBy_C&D', ...])])

In [75]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
import numpy as np
import pandas as pd

# [Previous preprocessing code stays the same until the transform part]

# Transform the data
X_transformed = preprocessor.transform(X)

# Get feature names after transformation
def get_feature_names(preprocessor):
    """Get feature names from preprocessor"""
    feature_names = []

    # For each transformer
    for name, transformer, features in preprocessor.transformers_:
        if name != 'remainder':
            if transformer == 'passthrough':
                # For binary variables that are passed through
                feature_names.extend(features)
            else:
                # For transformed variables
                feature_names.extend(features)

    return feature_names

# Get feature names
feature_names = get_feature_names(preprocessor)

# Create DataFrame
X_transformed_df = pd.DataFrame(
    X_transformed,
    columns=feature_names,
    index=X.index
)

X_transformed_df_ID = pd.merge(X[['FIPS_2020', 'Year']], X_transformed_df, left_index=True, right_index=True)
X_transformed_df_ID
# Now make sure the columns in X_transformed_df follow the same order as in X:
X_transformed_df_ID = X_transformed_df_ID[list(X.columns)]
X_transformed_df_ID

,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
1,2000,S06037,0.242170,0.001574,0.016090,0.000425,1.721991,1.0,0.479,0.074,...,0.0,-0.130764,1.958786,3.594304,0.006147,0.0,0.0,0.188,0.001285,0.004027
2,2000,S34001,0.239891,0.000351,0.020345,0.000702,-0.113515,1.0,0.663,0.055,...,0.0,1.934666,-0.194688,-0.183128,0.012680,0.0,0.0,0.346,0.000879,0.002085
3,2000,S34003,0.239891,0.000351,0.020345,0.000702,-0.025996,1.0,0.672,0.047,...,0.0,1.934666,-0.017380,-0.085427,0.012680,0.0,0.0,0.262,0.000879,0.002085
4,2000,S34005,0.239891,0.000351,0.020345,0.000702,-0.082119,1.0,0.774,0.053,...,0.0,1.934666,-0.150205,-0.152275,0.012680,0.0,0.0,0.311,0.000879,0.002085
5,2000,S34007,0.239891,0.000351,0.020345,0.000702,-0.134386,1.0,0.700,0.056,...,0.0,1.934666,-0.166234,-0.122450,0.012680,0.0,0.0,0.321,0.000879,0.002085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.238576,0.001352,0.021517,0.000446,-0.129705,1.0,0.303,0.060,...,0.0,-0.568885,0.050337,0.346515,0.006155,0.0,0.0,0.838,0.001512,0.000722
2469,2022,S36061,0.238576,0.001352,0.021517,0.000446,-0.016280,1.0,0.243,0.066,...,0.0,-0.568885,0.570021,7.324428,0.006155,0.0,0.0,0.885,0.001512,0.000722
2470,2022,S36081,0.238576,0.001352,0.021517,0.000446,0.406002,1.0,0.452,0.057,...,0.0,-0.568885,-0.022102,-0.167701,0.006155,0.0,0.0,0.828,0.001512,0.000722
2471,2022,S36085,0.238576,0.001352,0.021517,0.000446,-0.196978,1.0,0.686,0.061,...,0.0,-0.568885,-0.192078,-0.121422,0.006155,0.0,0.0,0.887,0.001512,0.000722


In [76]:
X

,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
1,2000,S06037,0.242170,0.001574,0.016090,0.000425,15007.942937,1.0,0.479,0.074,...,0.0,1.40,35657.0,734.600003,0.006147,0.0,0.0,0.188,0.001285,0.004027
2,2000,S34001,0.239891,0.000351,0.020345,0.000702,929.185718,1.0,0.663,0.055,...,0.0,8.00,994.0,0.000000,0.012680,0.0,0.0,0.346,0.000879,0.002085
3,2000,S34003,0.239891,0.000351,0.020345,0.000702,1600.476204,1.0,0.672,0.047,...,0.0,8.00,3848.0,19.000000,0.012680,0.0,0.0,0.262,0.000879,0.002085
4,2000,S34005,0.239891,0.000351,0.020345,0.000702,1170.000004,1.0,0.774,0.053,...,0.0,8.00,1710.0,6.000000,0.012680,0.0,0.0,0.311,0.000879,0.002085
5,2000,S34007,0.239891,0.000351,0.020345,0.000702,769.100006,1.0,0.700,0.056,...,0.0,8.00,1452.0,11.800000,0.012680,0.0,0.0,0.321,0.000879,0.002085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.238576,0.001352,0.021517,0.000446,805.000000,1.0,0.303,0.060,...,0.0,0.00,4938.0,103.000000,0.006155,0.0,0.0,0.838,0.001512,0.000722
2469,2022,S36061,0.238576,0.001352,0.021517,0.000446,1675.000000,1.0,0.243,0.066,...,0.0,0.00,13303.0,1460.000000,0.006155,0.0,0.0,0.885,0.001512,0.000722
2470,2022,S36081,0.238576,0.001352,0.021517,0.000446,4914.000000,1.0,0.452,0.057,...,0.0,0.00,3772.0,3.000000,0.006155,0.0,0.0,0.828,0.001512,0.000722
2471,2022,S36085,0.238576,0.001352,0.021517,0.000446,289.000000,1.0,0.686,0.061,...,0.0,0.00,1036.0,12.000000,0.006155,0.0,0.0,0.887,0.001512,0.000722


In [77]:
X.describe()

,Year,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,CBP: Food and Beverage Retailers (445),...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
count,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,...,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000
mean,2011.309312,0.240636,0.006355,0.019928,0.001170,1799.870081,0.934818,0.690512,0.066763,8100.467862,...,0.002024,1.817850,4127.753846,35.613037,0.009439,0.003239,0.235628,0.760240,0.002113,0.002866
std,6.034989,0.001792,0.007017,0.001450,0.000438,7671.784943,0.246897,0.127888,0.021451,27264.888012,...,0.044956,3.196108,16099.578587,194.510129,0.005229,0.056830,0.424476,0.155650,0.001011,0.003230
min,2000.000000,0.237614,0.000351,0.015169,0.000203,0.000000,0.000000,0.190000,0.019000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117000,0.000298,0.000529
25%,2006.000000,0.238872,0.001605,0.018610,0.000900,54.250000,1.000000,0.648168,0.056000,226.535714,...,0.000000,0.000000,50.000000,0.000000,0.005447,0.000000,0.000000,0.628394,0.001430,0.001246
50%,2012.000000,0.240820,0.003293,0.019790,0.001000,352.000000,1.000000,0.723000,0.064000,1797.069597,...,0.000000,0.000000,700.000000,0.000000,0.008149,0.000000,0.000000,0.810000,0.001537,0.002024
75%,2017.000000,0.242839,0.007339,0.021235,0.001354,1040.750000,1.000000,0.772769,0.072000,5736.882789,...,0.000000,1.500000,2491.000000,2.000000,0.012450,0.000000,0.000000,0.889000,0.003245,0.002494
max,2022.000000,0.243069,0.044351,0.026856,0.002555,133230.534776,1.000000,0.904000,0.257000,385721.000000,...,1.000000,8.000000,246256.000000,3475.000000,0.058368,1.000000,1.000000,0.968000,0.004230,0.016554


In [78]:
X_transformed_df_ID.describe()

,Year,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,CBP: Food and Beverage Retailers (445),...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
count,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2.470000e+03,2470.000000,2470.000000,2470.000000,2470.000000,...,2470.000000,2.470000e+03,2.470000e+03,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000,2470.000000
mean,2011.309312,0.240636,0.006355,0.019928,0.001170,1.006842e-17,0.934818,0.690512,0.066763,0.000000,...,0.002024,-9.205412e-17,-5.753382e-18,0.000000,0.009439,0.003239,0.235628,0.760240,0.002113,0.002866
std,6.034989,0.001792,0.007017,0.001450,0.000438,1.000202e+00,0.246897,0.127888,0.021451,1.000202,...,0.044956,1.000202e+00,1.000202e+00,1.000202,0.005229,0.056830,0.424476,0.155650,0.001011,0.003230
min,2000.000000,0.237614,0.000351,0.015169,0.000203,-2.346565e-01,0.000000,0.190000,0.019000,-0.297163,...,0.000000,-5.688851e-01,-2.564409e-01,-0.183128,0.000000,0.000000,0.000000,0.117000,0.000298,0.000529
25%,2006.000000,0.238872,0.001605,0.018610,0.000900,-2.275837e-01,1.000000,0.648168,0.056000,-0.288852,...,0.000000,-5.688851e-01,-2.533346e-01,-0.183128,0.005447,0.000000,0.000000,0.628394,0.001430,0.001246
50%,2012.000000,0.240820,0.003293,0.019790,0.001000,-1.887648e-01,1.000000,0.723000,0.064000,-0.231238,...,0.000000,-5.688851e-01,-2.129527e-01,-0.183128,0.008149,0.000000,0.000000,0.810000,0.001537,0.002024
75%,2017.000000,0.242839,0.007339,0.021235,0.001354,-9.896964e-02,1.000000,0.772769,0.072000,-0.086707,...,0.000000,-9.946928e-02,-1.016850e-01,-0.172844,0.012450,0.000000,0.000000,0.889000,0.003245,0.002494
max,2022.000000,0.243069,0.044351,0.026856,0.002555,1.713516e+01,1.000000,0.904000,0.257000,13.852872,...,1.000000,1.934666e+00,1.504246e+01,17.685883,0.058368,1.000000,1.000000,0.968000,0.004230,0.016554


### Apply on full dataset (predictors out of sample)

In [79]:
# Keep only variable in X_transformed_df that are used int he model:
df_full_sub = df_full[list(X.columns)]
df_full_sub

,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
1,2000,S01000,0.241354,0.004601,0.021205,0.001406,16482.319121,NaN,0.725065,0.069224,...,NaN,0.0,10736.0,250.400003,0.054233,NaN,NaN,0.303293,0.000261,0.005355
2,2000,S01001,0.241354,0.004601,0.021205,0.001406,113.385714,NaN,0.808000,0.057000,...,NaN,0.0,137.0,0.000000,0.054233,NaN,NaN,0.338000,0.000261,0.005355
3,2000,S01003,0.241354,0.004601,0.021205,0.001406,707.190479,NaN,0.796000,0.049000,...,NaN,0.0,885.0,0.000000,0.054233,NaN,NaN,0.296000,0.000261,0.005355
4,2000,S01005,0.241354,0.004601,0.021205,0.001406,38.500000,NaN,0.732000,0.068000,...,NaN,0.0,37.0,0.000000,0.054233,NaN,NaN,0.324000,0.000261,0.005355
5,2000,S01007,0.241354,0.004601,0.021205,0.001406,4.000000,NaN,0.802000,0.069000,...,NaN,0.0,9.0,0.000000,0.054233,NaN,NaN,0.357000,0.000261,0.005355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73478,2022,S56037,0.240103,0.003505,0.016434,0.000656,163.000000,NaN,0.734000,0.061000,...,NaN,0.0,61.0,0.000000,0.000000,NaN,NaN,0.930000,0.001733,0.000978
73479,2022,S56039,0.240103,0.003505,0.016434,0.000656,60.000000,NaN,0.622000,0.042000,...,NaN,0.0,2461.0,0.000000,0.000000,NaN,NaN,0.960000,0.001733,0.000978
73480,2022,S56041,0.240103,0.003505,0.016434,0.000656,37.000000,NaN,0.780000,0.061000,...,NaN,0.0,42.0,0.000000,0.000000,NaN,NaN,0.944000,0.001733,0.000978
73481,2022,S56043,0.240103,0.003505,0.016434,0.000656,77.000000,NaN,0.716000,0.047000,...,NaN,0.0,0.0,0.000000,0.000000,NaN,NaN,0.947000,0.001733,0.000978


In [80]:
df_full_sub.isna().sum().sort_values(ascending=False)

,0
ActivityProducedBy_Industrial,71074
ActivityProducedBy_Landscape,71074
ActivityConsumedBy_Recycling,71074
ActivityProducedBy_C&D,71074
Yard Waste Included,71074
...,...
CBP: Insurance Carriers and Related Activities (524),0
CBP: Transportation Equipment Manufacturing (336),0
PCE: Clothing and footwear,0
FIPS_2020,0


#### Fill in measurement-related values to the desired prediction:

These depend on the measurement, so we choose them depending on what we want the prediction to represent. They aim at capturing differences in the studies that are based on measurement method/location/activity instead of generation in a given location-year.

In [81]:
df_cleaned_sub = df_full_sub.copy()

# ActivityConsumedBy columns
df_cleaned_sub['ActivityConsumedBy_Landfill'] = 1
df_cleaned_sub['ActivityConsumedBy_Recycling'] = 1

df_cleaned_sub['ActivityConsumedBy_Compost'] = 0
df_cleaned_sub['ActivityConsumedBy_Incinerator'] = 0
df_cleaned_sub['ActivityConsumedBy_MRF'] = 0
df_cleaned_sub['ActivityConsumedBy_Unknown'] = 0

df_cleaned_sub['ActivityProducedBy_C&D'] = 0
df_cleaned_sub['ActivityProducedBy_Landscape'] = 0
df_cleaned_sub['ActivityProducedBy_Unspecified MSW'] = 0

df_cleaned_sub['ActivityProducedBy_Commercial'] = 0
df_cleaned_sub['ActivityProducedBy_Industrial'] = 0
df_cleaned_sub['ActivityProducedBy_Institutional'] = 0
df_cleaned_sub['ActivityProducedBy_Residential'] = 0

# Inclusion columns
df_cleaned_sub['Recycling Included'] = 1
df_cleaned_sub['Yard Waste Included'] = 1
df_cleaned_sub['Food Waste Included'] = 1
df_cleaned_sub['Disposal Included'] = 1
df_cleaned_sub['Yard Waste Included Unknown'] = 0
df_cleaned_sub['Food Waste Included Unknown'] = 0

# PointOfMeasurement columns
df_cleaned_sub['PointOfMeasurement_Compost'] = 0
df_cleaned_sub['PointOfMeasurement_Composting'] = 0
df_cleaned_sub['PointOfMeasurement_Curbside Collection'] = 0
df_cleaned_sub['PointOfMeasurement_Incinerator/Combustion'] = 0
df_cleaned_sub['PointOfMeasurement_Landfill'] = 0
df_cleaned_sub['PointOfMeasurement_MRF'] = 0
df_cleaned_sub['PointOfMeasurement_Material Recovery Facility'] = 0
df_cleaned_sub['PointOfMeasurement_Recycling'] = 0
df_cleaned_sub['PointOfMeasurement_Self-Haul'] = 0
df_cleaned_sub['PointOfMeasurement_Transfer Station'] = 0
df_cleaned_sub['PointOfMeasurement_Unspecified Waste Facility'] = 0
df_cleaned_sub['PointOfMeasurement_Waste Haulers'] = 0

df_cleaned_sub.isna().sum().sort_values(ascending=False)

,0
IP: Federal general government (nondefense - GFGN),46
IP: State and local general government (GSLG),46
"IP: Federal Reserve banks, credit intermediation, and related activities",46
IP: Social assistance,46
IP: Oil and gas extraction,46
...,...
"CBP: Mining, Quarrying, and Oil and Gas Extraction (21)",0
CBP: Lessors of Nonfinancial Intangible Assets (except Copyrighted Works) (533),0
"CBP: Finance and Insurance (52 excluding 525110, 525120, 525190, 525920)",0
CBP: Real Estate and Rental and Leasing (53),0


In [82]:
df_cleaned_sub[df_cleaned_sub['IP: Transit and ground passenger transportation'].isna()]
# Still a few missing points remain but it is only two location that are missing throughout all years (FIPS 11000 and 11001 - DC and Montgomery, Alabama)

,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
329,2000,S11000,0.241907,NaN,0.012732,NaN,621.357895,0,0.408000,0.091,...,0,NaN,1690.324561,164.292281,NaN,0,0,0.206000,NaN,NaN
330,2000,S11001,0.241907,NaN,0.012732,NaN,621.357895,0,0.408000,0.091,...,0,NaN,1690.324561,164.292281,NaN,0,0,0.206000,NaN,NaN
3525,2001,S11000,0.241343,NaN,0.013172,NaN,698.589474,0,0.419176,0.090,...,0,NaN,1762.947368,157.435790,NaN,0,0,0.466009,NaN,NaN
3526,2001,S11001,0.241343,NaN,0.013172,NaN,698.589474,0,0.419176,0.090,...,0,NaN,1762.947368,157.435790,NaN,0,0,0.466009,NaN,NaN
6720,2002,S11000,0.240958,NaN,0.013204,NaN,775.821053,0,0.419110,0.089,...,0,NaN,1835.570175,150.579299,NaN,0,0,0.503249,NaN,NaN
6721,2002,S11001,0.240958,NaN,0.013204,NaN,775.821053,0,0.419110,0.089,...,0,NaN,1835.570175,150.579299,NaN,0,0,0.503249,NaN,NaN
9916,2003,S11000,0.240290,NaN,0.013335,NaN,853.052632,0,0.419045,0.089,...,0,NaN,1908.192982,143.722807,NaN,0,0,0.540453,NaN,NaN
9917,2003,S11001,0.240290,NaN,0.013335,NaN,853.052632,0,0.419045,0.089,...,0,NaN,1908.192982,143.722807,NaN,0,0,0.540453,NaN,NaN
13116,2004,S11000,0.240306,NaN,0.013589,NaN,1081.000000,0,0.418979,0.091,...,0,NaN,2011.000000,90.800001,NaN,0,0,0.577212,NaN,NaN
13117,2004,S11001,0.240306,NaN,0.013589,NaN,1081.000000,0,0.418979,0.091,...,0,NaN,2011.000000,90.800001,NaN,0,0,0.577212,NaN,NaN


In [83]:
# df_cleaned_sub.to_excel(folder_path_OUTPUT_VARIABLES + f"ModelInput_AllCountiesStates_{name_suffix}.xlsx", index=False)
print("Model saved to : ", folder_path_OUTPUT_VARIABLES + f"ModelInput_AllCountiesStates_{name_suffix}.xlsx")

Model saved to :  /content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/ModelInput_AllCountiesStates_KeepImp0_GenInter0_SmallSample0.xlsx


In [84]:
# Transform the data
X_transformed = preprocessor.transform(df_cleaned_sub)

# Create DataFrame
df_predictors_transformed = pd.DataFrame(
    X_transformed,
    columns=feature_names,
    index=df_cleaned_sub.index
)

df_predictors_transformed_ID = pd.merge(df_cleaned_sub[['FIPS_2020', 'Year']], df_predictors_transformed, left_index=True, right_index=True)
df_predictors_transformed_ID


,FIPS_2020,Year,CBP: Building Material and Garden Equipment and Supplies Dealers (444),CBP: Administrative and Support Services (561),POL: Tipping Fee,"CBP: Other Services Except Public Administration (81, excl. 814)",CBP: Heavy and Civil Engineering Construction (237),CBP: Hospitals (622),CBP: Motor Vehicle and Parts Dealers (441),CBP: Utilities (221),...,ActivityProducedBy_C&D,POL: Require curbside recycling,Recycling Included,PointOfMeasurement_Compost,ActivityProducedBy_Commercial,ActivityConsumedBy_Compost,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,ActivityConsumedBy_Landfill,Disposal Included
1,S01000,2000,1.605368,1.212869,-1.187967,1.463671,1.914212,1.502910,1.629294,2.649565,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,S01001,2000,-0.268167,-0.278517,-1.187967,-0.282943,-0.219874,-0.317694,-0.249653,-0.267883,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,S01003,2000,-0.198871,-0.242681,-1.187967,-0.244816,-0.142457,-0.271629,-0.207346,-0.241827,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,S01005,2000,-0.269577,-0.272988,-1.187967,-0.294768,-0.229637,-0.313859,-0.249518,-0.260410,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
5,S01007,2000,-0.276442,-0.280433,-1.187967,-0.295610,-0.234135,-0.313859,-0.260514,-0.279396,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73478,S56037,2022,-0.252561,-0.274436,0.296063,-0.288793,-0.213406,-0.317694,-0.241332,-0.184464,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
73479,S56039,2022,-0.266580,-0.267722,0.296063,-0.283974,-0.226834,-0.317694,-0.248235,-0.284850,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
73480,S56041,2022,-0.273542,-0.279905,0.296063,-0.297378,-0.229833,-0.317694,-0.255688,-0.275356,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
73481,S56043,2022,-0.270061,-0.280013,0.296063,-0.296809,-0.224618,-0.317694,-0.260820,-0.283032,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [85]:
# Now make sure the columns in X_transformed_df follow the same order as in X:
df_predictors_transformed_ID = df_predictors_transformed_ID[list(df_cleaned_sub.columns)]
df_predictors_transformed_ID


,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
1,2000,S01000,0.241354,0.004601,0.021205,0.001406,1.914212,0.0,0.725065,0.069224,...,0.0,-0.568885,0.410544,1.104469,0.054233,0.0,0.0,0.303293,0.000261,0.005355
2,2000,S01001,0.241354,0.004601,0.021205,0.001406,-0.219874,0.0,0.808000,0.057000,...,0.0,-0.568885,-0.247930,-0.183128,0.054233,0.0,0.0,0.338000,0.000261,0.005355
3,2000,S01003,0.241354,0.004601,0.021205,0.001406,-0.142457,0.0,0.796000,0.049000,...,0.0,-0.568885,-0.201459,-0.183128,0.054233,0.0,0.0,0.296000,0.000261,0.005355
4,2000,S01005,0.241354,0.004601,0.021205,0.001406,-0.229637,0.0,0.732000,0.068000,...,0.0,-0.568885,-0.254142,-0.183128,0.054233,0.0,0.0,0.324000,0.000261,0.005355
5,2000,S01007,0.241354,0.004601,0.021205,0.001406,-0.234135,0.0,0.802000,0.069000,...,0.0,-0.568885,-0.255882,-0.183128,0.054233,0.0,0.0,0.357000,0.000261,0.005355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73478,2022,S56037,0.240103,0.003505,0.016434,0.000656,-0.213406,0.0,0.734000,0.061000,...,0.0,-0.568885,-0.252651,-0.183128,0.000000,0.0,0.0,0.930000,0.001733,0.000978
73479,2022,S56039,0.240103,0.003505,0.016434,0.000656,-0.226834,0.0,0.622000,0.042000,...,0.0,-0.568885,-0.103549,-0.183128,0.000000,0.0,0.0,0.960000,0.001733,0.000978
73480,2022,S56041,0.240103,0.003505,0.016434,0.000656,-0.229833,0.0,0.780000,0.061000,...,0.0,-0.568885,-0.253832,-0.183128,0.000000,0.0,0.0,0.944000,0.001733,0.000978
73481,2022,S56043,0.240103,0.003505,0.016434,0.000656,-0.224618,0.0,0.716000,0.047000,...,0.0,-0.568885,-0.256441,-0.183128,0.000000,0.0,0.0,0.947000,0.001733,0.000978


# Saving final output:

In [86]:
X.to_excel(folder_path_OUTPUT_VARIABLES + f"ModelInput_X_{name_suffix}.xlsx", index=False)
print("Model saved to : ", folder_path_OUTPUT_VARIABLES + f"ModelInput_X_{name_suffix}.xlsx")
X

Model saved to :  /content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/ModelInput_X_KeepImp0_GenInter0_SmallSample0.xlsx


,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
1,2000,S06037,0.242170,0.001574,0.016090,0.000425,15007.942937,1.0,0.479,0.074,...,0.0,1.40,35657.0,734.600003,0.006147,0.0,0.0,0.188,0.001285,0.004027
2,2000,S34001,0.239891,0.000351,0.020345,0.000702,929.185718,1.0,0.663,0.055,...,0.0,8.00,994.0,0.000000,0.012680,0.0,0.0,0.346,0.000879,0.002085
3,2000,S34003,0.239891,0.000351,0.020345,0.000702,1600.476204,1.0,0.672,0.047,...,0.0,8.00,3848.0,19.000000,0.012680,0.0,0.0,0.262,0.000879,0.002085
4,2000,S34005,0.239891,0.000351,0.020345,0.000702,1170.000004,1.0,0.774,0.053,...,0.0,8.00,1710.0,6.000000,0.012680,0.0,0.0,0.311,0.000879,0.002085
5,2000,S34007,0.239891,0.000351,0.020345,0.000702,769.100006,1.0,0.700,0.056,...,0.0,8.00,1452.0,11.800000,0.012680,0.0,0.0,0.321,0.000879,0.002085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.238576,0.001352,0.021517,0.000446,805.000000,1.0,0.303,0.060,...,0.0,0.00,4938.0,103.000000,0.006155,0.0,0.0,0.838,0.001512,0.000722
2469,2022,S36061,0.238576,0.001352,0.021517,0.000446,1675.000000,1.0,0.243,0.066,...,0.0,0.00,13303.0,1460.000000,0.006155,0.0,0.0,0.885,0.001512,0.000722
2470,2022,S36081,0.238576,0.001352,0.021517,0.000446,4914.000000,1.0,0.452,0.057,...,0.0,0.00,3772.0,3.000000,0.006155,0.0,0.0,0.828,0.001512,0.000722
2471,2022,S36085,0.238576,0.001352,0.021517,0.000446,289.000000,1.0,0.686,0.061,...,0.0,0.00,1036.0,12.000000,0.006155,0.0,0.0,0.887,0.001512,0.000722


In [87]:
X_transformed_df_ID.to_excel(folder_path_OUTPUT_VARIABLES + f"ModelInput_X_transformed_df_{name_suffix}.xlsx", index=False)
print("Model saved to : ", folder_path_OUTPUT_VARIABLES + f"ModelInput_X_transformed_df_{name_suffix}.xlsx")
X_transformed_df_ID

Model saved to :  /content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/ModelInput_X_transformed_df_KeepImp0_GenInter0_SmallSample0.xlsx


,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
1,2000,S06037,0.242170,0.001574,0.016090,0.000425,1.721991,1.0,0.479,0.074,...,0.0,-0.130764,1.958786,3.594304,0.006147,0.0,0.0,0.188,0.001285,0.004027
2,2000,S34001,0.239891,0.000351,0.020345,0.000702,-0.113515,1.0,0.663,0.055,...,0.0,1.934666,-0.194688,-0.183128,0.012680,0.0,0.0,0.346,0.000879,0.002085
3,2000,S34003,0.239891,0.000351,0.020345,0.000702,-0.025996,1.0,0.672,0.047,...,0.0,1.934666,-0.017380,-0.085427,0.012680,0.0,0.0,0.262,0.000879,0.002085
4,2000,S34005,0.239891,0.000351,0.020345,0.000702,-0.082119,1.0,0.774,0.053,...,0.0,1.934666,-0.150205,-0.152275,0.012680,0.0,0.0,0.311,0.000879,0.002085
5,2000,S34007,0.239891,0.000351,0.020345,0.000702,-0.134386,1.0,0.700,0.056,...,0.0,1.934666,-0.166234,-0.122450,0.012680,0.0,0.0,0.321,0.000879,0.002085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2022,S36047,0.238576,0.001352,0.021517,0.000446,-0.129705,1.0,0.303,0.060,...,0.0,-0.568885,0.050337,0.346515,0.006155,0.0,0.0,0.838,0.001512,0.000722
2469,2022,S36061,0.238576,0.001352,0.021517,0.000446,-0.016280,1.0,0.243,0.066,...,0.0,-0.568885,0.570021,7.324428,0.006155,0.0,0.0,0.885,0.001512,0.000722
2470,2022,S36081,0.238576,0.001352,0.021517,0.000446,0.406002,1.0,0.452,0.057,...,0.0,-0.568885,-0.022102,-0.167701,0.006155,0.0,0.0,0.828,0.001512,0.000722
2471,2022,S36085,0.238576,0.001352,0.021517,0.000446,-0.196978,1.0,0.686,0.061,...,0.0,-0.568885,-0.192078,-0.121422,0.006155,0.0,0.0,0.887,0.001512,0.000722


In [88]:
df_predictors_transformed_ID.to_excel(folder_path_OUTPUT_VARIABLES + f"ModelInput_predictors_AllStatesCountiesYears_df_{name_suffix}.xlsx", index=False)
print("Model saved to : ", folder_path_OUTPUT_VARIABLES + f"ModelInput_predictors_AllStatesCountiesYears_df_{name_suffix}.xlsx")
df_predictors_transformed_ID

Model saved to :  /content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/ModelInput_predictors_AllStatesCountiesYears_df_KeepImp0_GenInter0_SmallSample0.xlsx


,Year,FIPS_2020,PCE: Personal consumption expenditures,IP: Wood products,PCE: Other nondurable goods,IP: Warehousing and storage,CBP: Heavy and Civil Engineering Construction (237),Yard Waste Included Unknown,DEM: Owner-Occupied Housing,DEM: 20-24 years,...,ActivityProducedBy_Landscape,POL: Landfill tax,"CBP: Amusement, Gambling, and Recreation Industries (713)","CBP: Funds, Trusts, and Other Financial Vehicles (525)",IP: Paper products,PointOfMeasurement_Transfer Station,ActivityProducedBy_Institutional,DEM: High School Degree,"IP: Amusements, gambling, and recreation industries",IP: Federal general government (nondefense - GFGN)
1,2000,S01000,0.241354,0.004601,0.021205,0.001406,1.914212,0.0,0.725065,0.069224,...,0.0,-0.568885,0.410544,1.104469,0.054233,0.0,0.0,0.303293,0.000261,0.005355
2,2000,S01001,0.241354,0.004601,0.021205,0.001406,-0.219874,0.0,0.808000,0.057000,...,0.0,-0.568885,-0.247930,-0.183128,0.054233,0.0,0.0,0.338000,0.000261,0.005355
3,2000,S01003,0.241354,0.004601,0.021205,0.001406,-0.142457,0.0,0.796000,0.049000,...,0.0,-0.568885,-0.201459,-0.183128,0.054233,0.0,0.0,0.296000,0.000261,0.005355
4,2000,S01005,0.241354,0.004601,0.021205,0.001406,-0.229637,0.0,0.732000,0.068000,...,0.0,-0.568885,-0.254142,-0.183128,0.054233,0.0,0.0,0.324000,0.000261,0.005355
5,2000,S01007,0.241354,0.004601,0.021205,0.001406,-0.234135,0.0,0.802000,0.069000,...,0.0,-0.568885,-0.255882,-0.183128,0.054233,0.0,0.0,0.357000,0.000261,0.005355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73478,2022,S56037,0.240103,0.003505,0.016434,0.000656,-0.213406,0.0,0.734000,0.061000,...,0.0,-0.568885,-0.252651,-0.183128,0.000000,0.0,0.0,0.930000,0.001733,0.000978
73479,2022,S56039,0.240103,0.003505,0.016434,0.000656,-0.226834,0.0,0.622000,0.042000,...,0.0,-0.568885,-0.103549,-0.183128,0.000000,0.0,0.0,0.960000,0.001733,0.000978
73480,2022,S56041,0.240103,0.003505,0.016434,0.000656,-0.229833,0.0,0.780000,0.061000,...,0.0,-0.568885,-0.253832,-0.183128,0.000000,0.0,0.0,0.944000,0.001733,0.000978
73481,2022,S56043,0.240103,0.003505,0.016434,0.000656,-0.224618,0.0,0.716000,0.047000,...,0.0,-0.568885,-0.256441,-0.183128,0.000000,0.0,0.0,0.947000,0.001733,0.000978


In [89]:
y.to_excel(folder_path_OUTPUT_VARIABLES + f"ModelInput_y_{name_suffix}.xlsx", index=False)
print("Model saved to : ", folder_path_OUTPUT_VARIABLES + f"ModelInput_y_{name_suffix}.xlsx")
y

Model saved to :  /content/drive/MyDrive/Waste_Prediction_Outputs/ModelingDatasets/02_04_2025/ModelInput_y_KeepImp0_GenInter0_SmallSample0.xlsx


,Year,FIPS_2020,Waste Generation Amount
1,2000,S06037,11384354.0
2,2000,S34001,329432.0
3,2000,S34003,1040243.0
4,2000,S34005,469772.0
5,2000,S34007,493305.0
...,...,...,...
2468,2022,S36047,831039.0
2469,2022,S36061,395511.3
2470,2022,S36081,711611.3
2471,2022,S36085,182057.8


# END

In [90]:
# Get the current time in UTC and convert it to Central Time
current_time_central = datetime.now(central_time)
print("Central Time:", current_time_central.strftime('%Y-%m-%d %H:%M:%S %Z'))

Central Time: 2025-02-10 08:37:20 CST
